# Edema study

This notebook plots figures for the edema related studies

## Imports

In [ ]:
from typing import Mapping, Any, List, Tuple, Union, Optional, Callable
from numpy.typing import NDArray

# Standard library
import sys
import itertools
from os import path

# Utilities
from tqdm.notebook import tqdm
from pprint import pprint

# Numerical
import numpy as np
import scipy.signal as signal
import h5py

# FEM
import dolfin as dfn
import ufl

# Plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import tri

import pyvista as pv
pv.start_xvfb()

# My libraries
from femvf import forward, statefile as sf, meshutils
from femvf.models.dynamical import (solid as dsolid, fluid as dfluid)
from femvf.load import load_dynamical_fsi_model

from blockarray import blockvec

from vfsig import modal, fftutils, clinical

from exputils import h5utils

from experiment import setup, solve, post
import viciouscycle as vc
import cases

sys.path.append('../../vf-onset-sensitivity')
import libhopf

dfn.set_log_level(50)

## Constants

In [ ]:
## Set the default simulation case
# (you may have to change these depending on the run)

ExpParam = cases.ExpParam

DT = 1.25e-5
TF = 0.5

DT = 5e-5
TF = 0.5

ECOV = 2.5e3 * 10
EBOD = 5e3 * 10
PSUB = 0.3e3 * 10

VCOVS = np.array([1, 1.05, 1.1, 1.15, 1.2, 1.25, 1.3])
MCOVS = np.array([0, -0.4, -0.8, -1.2, -1.6])

VCOVS = np.array([1, 1.025, 1.05, 1.075, 1.1])
MCOVS = np.array([0.0])

OUT_DIR = 'out_debug_dt_psub'
OUT_DIR = 'out'
POST_DIR = OUT_DIR

DEFAULT_CASE_PARAM = ExpParam({
    'MeshName': 'M5_BC',
    'GA': 3.0,
    'clscale': 0.75,
    'DZ': 1.50, 'NZ': 15,
    'Ecov': ECOV,
    'Ebod': EBOD,
    'psub': PSUB,
    'vcov': 1.0,
    'mcov': -0.8,
    'dt': DT,
    'tf': TF,
    'ModifyEffect': '',
    'SwellingDistribution': 'field.tavg_viscous_rate',
    'SwellingModel': 'power'
})

In [ ]:
DEFAULT_CASE_PARAM.to_str()

## Functions

### Loading results

In [ ]:
def get_casenames(data: Mapping[str, Any]):
    """Return all the case name strings in a dictionary of results"""

    casenames = set([key.split('/')[0] for key in data.keys()])
    return list(casenames)

def get_resultnames(data: Mapping[str, Any]):
    """Return all the result name strings in a dictionary of results"""

    resultnames = set(['/'.join(key.split('/')[1:]) for key in data.keys()])
    return list(resultnames)

In [ ]:
## Functions for accessing post-processed results

_DEFAULT_CASE_PARAM = DEFAULT_CASE_PARAM

def load_dataset(
        data: Mapping[str, Any],
        dataset_name: str,
        case_param: ExpParam,
        sub: Optional[Mapping[str, Any]]=None
    ) -> NDArray:
    """
    Return an array dataset

    Parameters
    ----------
    data: Mapping[str, Any]
        A mapping from result names to their values
    measure_name: str
        A measure name to load
    case_param: List[Mapping[str, Any]]
        A mapping from parameter labels to parameter value(s). Measurements
        are extracted from these parameters.
    sub: Optional[Mapping[str, Any]]
        A parameter to substitute in `case_param`
    """
    if sub is None:
        sub = {}

    if isinstance(case_param, ExpParam):
        result = data[f'{case_param.substitute(sub).to_str()}/{dataset_name}']
    else:
        raise TypeError(f'Invalid type for `case_param`, {type(case_param)}')
    return result

def load_datasets(
        data: Mapping[str, Any],
        dataset_name: str,
        case_params: List[ExpParam],
        sub: Optional[Mapping[str, Any]]=None
    ) -> List[NDArray]:
    """
    Return a series of scalar datasets

    Parameters
    ----------
    data: Mapping[str, Any]
        A mapping from result names to their values
    measure_name: str
        A measure name to load
    case_params: List[ExpParam]
        A mapping from parameter labels to parameter value(s). Measurements
        are extracted from these parameters.
    sub: Optional[Mapping[str, Any]]
        A parameter to substitute in `case_param`
    """
    return [load_dataset(data, dataset_name, case_param, sub=sub) for case_param in case_params]


In [ ]:
MESH_DIR = '../mesh'
class ModelLoader:
    """
    Load models from a cache

    This speeds up model loading in case a model with the same mesh has already 
    been loaded.
    """

    def __init__(self, models: Mapping[str, Any]=None):
        if models is None:
            models = {}
        self._models = models

    def __call__(self, param: ExpParam):
        mesh_name = setup.setup_mesh_name(param)

        if mesh_name not in self._models:
            self._models[mesh_name] = setup.setup_model(param, mesh_dir=MESH_DIR)

        return self._models[mesh_name]

load_model = ModelLoader()

### Post-processing

In [ ]:
def postproc_acoustics_from_data(data, casenames=None):
    acoustics_data = {}

    if casenames is None:
        casenames = get_casenames(data)

    ts = [data[f'{casename}/time.t'] for casename in casenames]
    qs = [data[f'{casename}/time.q'] for casename in casenames]
    prmss = [post.calc_prms(t, q) for t, q in zip(ts, qs)]
    
    _data = {
        f'{casename}/prms': prms
        for casename, prms in zip(casenames, prmss)
    }
    acoustics_data.update(_data)
    
    _data = {
        f'{casename}/spl': 20*np.log10(prms/10/20e-6)
        for casename, prms in zip(casenames, prmss)
    }
    acoustics_data.update(_data)

    # Each element of `fund_mode_descrs` is `fund_freq, fund_phase, dfreq, dphase`
    fund_mode_descrs = [modal.fundamental_mode_from_rfft(q, t[1]-t[0])[:4] for q, t in zip(qs, ts)]
    
    _data = {
        f'{casename}/fund_freq': fund_mode_descr[0] 
        for casename, fund_mode_descr in zip(casenames, fund_mode_descrs)
    }
    acoustics_data.update(_data)
    
    _data = {
        f'{casename}/dfreq': fund_mode_descr[2] 
        for casename, fund_mode_descr in zip(casenames, fund_mode_descrs)
    }
    acoustics_data.update(_data)

    return acoustics_data

In [ ]:
def postproc_fieldavg_from_data(data, fspace, dx, fieldname, casenames=None):

    if casenames is None:
        casenames = get_casenames(data)
        casenames = [casename for casename in casenames if f'{casename}/{fieldname}' in data]

    fieldavg_data = {}

    # Define symbolic expressions needed for the field volume average
    ufl_f = dfn.Function(fspace)
    vol = dfn.assemble(1*dx)

    def field_avg(f):
        ufl_f.vector()[:] = f
        return dfn.assemble(ufl_f*dx)/vol

    fieldavg_data = {
        f'{casename}/cover_avg_{fieldname}': field_avg(DATA[f'{casename}/{fieldname}']) 
        for casename in casenames
    }
    return fieldavg_data

### Processing time signals

In [ ]:
def tavg(y, t, axis=-1):
    """
    Return the time average of a signal
    """
    tavg_y = np.trapz(y, x=t, axis=axis)/np.trapz(np.ones(t.shape), x=t, axis=axis)
    return tavg_y

### Processing the model

In [ ]:
def make_solve_static(model):
    """
    Return the static state for a solid model with no external loading
    """
    forms = model.solid.forms
    f1uva = forms['form.un.f1uva']
    u1 = forms['coeff.state.u1']
    bc_dir = forms['bc.dirichlet']

    def solve_static():
        dfn.solve(f1uva==0, u1, bcs=[bc_dir])
        x = model.state1.copy()
        x['v'][:] = 0
        x['a'][:] = 0
        return x
    return solve_static

In [ ]:
def get_dx(model, name=None):
    cell_label_to_id = model.solid.residual.mesh_function_label_to_value('cell')
    _dx = model.solid.residual.measure('dx')
    if name is None:
        dx = _dx
    elif name == 'cover':
        # dx = (
        #     _dx(int(cell_label_to_id['inferior']))
        #     +_dx(int(cell_label_to_id['medial']))
        #     + _dx(int(cell_label_to_id['superior']))
        # )
        dx = _dx(int(cell_label_to_id['cover']))
    else:
        dx = _dx(int(cell_label_to_id[name]))
    return dx

In [ ]:
def make_vtk_mesh_descr(mesh: dfn.Mesh):
    # `cells` is the `pyvista`/`vtk` mesh format;
    # the first entry in the cell is the number of points in the cell (4 for tetrahedron)
    cells = np.zeros((mesh.cells().shape[0], mesh.cells().shape[1]+1), dtype=int)
    cells[:, 1:] = mesh.cells()
    cells[:, 0] = 4
    
    cell_types = pv.CellType.TETRA*np.ones(cells.shape[0], dtype=int)
    
    points = mesh.coordinates()
    return cells, cell_types, points

### For plotting

In [ ]:
## Functions for plotting percent change
def per_change_forward(y, y0):
    return (y-y0)/y0 * 100

def per_change_inv(y, y0):
    return y*y0/100 + y0

In [ ]:
def annotate_vertical_trend(ax, text, x, y1, y2):
    """
    Annotate a trend from 'y1' to 'y2'
    """
    ax.annotate(
        text, (x, y2), xytext=(x, y1),  ha='center',
        textcoords='data', xycoords='data',
        arrowprops={'arrowstyle': '->'}
    )

In [ ]:
def plot_tripcolor_seq(axs, trian, zdata, zmin=None, zmax=None):
    """
    Plot a sequence of `tripcolor` plots with shared z limits

    By default, all plots are normalized between the min/max of all z data.

    Parameters
    ----------
    axs : mpl.axes.Axes
        An array of axes to plot in
    trian : mpl.triangulation.Triangulation
        The triangulation used for data
    zdata : List[np.ndarray]
        A list of z data for each tri-plot

    Returns
    -------
    List[mpl.artist.Artist]
        A list of artists for each `tripcolor` call. You can use any of the
        artists to create a colorbar since they have the same normalization.
    """
    if zmin is None:
        zmin = np.min(zdata)
    if zmax is None:
        zmax = np.max(zdata)
    kwargs = {'vmin': zmin, 'vmax': zmax, 'edgecolors': 'none'}

    fspace_cg1 = dfn.FunctionSpace(model.solid.residual.mesh(), 'CG', 1)
    fspace_dg0 = dfn.FunctionSpace(model.solid.residual.mesh(), 'DG', 0)
    z_cg1 = dfn.Function(fspace_cg1)
    z_dg0 = dfn.Function(fspace_dg0)

    zdata_flat = zdata.reshape(-1, zdata.shape[-1])
    for z, ax in zip(zdata_flat, axs.flat):
        artists = ax.tripcolor(trian, z, **kwargs)
        # ax.tricontour(trian, y, [0])

        ## Project the dg0 stress to CG1 so you can plot contours
        # print(y_dg0.vector().size(), y.shape)
        z_dg0.vector()[:] = z
        z_vertex = dfn.project(z_dg0, fspace_cg1, function=z_cg1).vector()[VERT_TO_SDOF]
        # Setting 'Greys' cmap and vmin/vmax appropriately will ensure a specific color for the contour line
        # cmap = plt.get_cmap(name='Greys')
        ax.tricontour(trian, z_vertex, [0], colors=['k'], linewidths=[1])
    return artists

In [ ]:
def plot_cell_scalar(
    plotter: pv.Plotter, 
    grid: pv.Grid, 
    cell_scalar: NDArray, 
    format_camera: Callable[[pv.Camera, pv.Grid, pv.Grid], None]=None, 
    clip_origin = None,
    clip_normal = None,
    **kwargs_add_mesh
):
    """
    Return an image of a clipped cell scalar for a mesh
    """
    # grid = pv.UnstructuredGrid(cells, cell_types, points)
    grid.cell_data['cell_scalar'] = cell_scalar

    if clip_origin is not None and clip_normal is not None:
        clipped_grid = grid.clip(normal=clip_normal, origin=clip_origin)
    
    plotter.add_mesh(clipped_grid, scalars='cell_scalar', show_edges=True, color=None, show_scalar_bar=False, **kwargs_add_mesh)

    camera = plotter.camera
    if format_camera is not None:
        format_camera(camera, grid, clipped_grid)
    else:
        camera.focal_point = grid.center
        camera.position = grid.center + 2.5*np.array((-0.25, 0.5, 1))
        camera.up = (0, 1, 0)
    
    img = plotter.screenshot()
    return img

In [ ]:
def normal(x):
    """
    Return the normal vector to a 2D curve `x`
    """
    assert x.ndim == 2
    assert x.shape[-1] == 2

    dx = np.zeros(x.shape)
    dx[1:, :] = x[1:] - x[:-1]
    s = np.cumsum(np.linalg.norm(dx, axis=-1))

    # Tangent
    dx = np.gradient(x[:, 0], s, axis=0)
    dy = np.gradient(x[:, 1], s, axis=0)

    # Non-unit normal
    nx = dy
    ny = -dx
    n = np.stack([nx, ny], axis=-1)
    return n/np.linalg.norm(n, axis=-1, keepdims=True)

def offset_normal(x, n):
    return x + n

In [ ]:
def format_axs_grid(axs):
    for ax in axs[:-1, :].flat:
        ax.tick_params('x', labelbottom=False)

    for ax in axs[:, 1:].flat:
        ax.tick_params('y', labelleft=False)

# def format_axis_loc

## Configure figures

In [ ]:
## Set any figure constants
FIG_TYPE = 'manuscript'
# FIG_TYPE = 'presentation'

CM = 1/2.54
if FIG_TYPE == 'manuscript':
    FIG_DIR = '../fig/manuscript'
    FIG_EXT = 'pdf'

    FIG_LX = 8.4*CM
    FIG_LX_WIDE = 17.4*CM

    FIG_LY = 23.4*CM
    FONTSIZE = 9
elif FIG_TYPE == 'presentation':
    FIG_DIR = '../fig/presentation'
    FIG_EXT = 'svg'

    FIG_LX = 7.5*CM
    FIG_LX_WIDE = 30*CM

    FIG_LY = 13*CM
    FONTSIZE = 14

SIZE_SMALL = (FIG_LX, FIG_LX)
SIZE_MED = (FIG_LX, 1.25*FIG_LX)
SIZE_MED_WIDE = (FIG_LX_WIDE, 1.25*FIG_LX)
SIZE_LARGE = (FIG_LX, 1.5*FIG_LX)
SIZE_LARGE_WIDE = (FIG_LX_WIDE, 1.5*FIG_LX)

DPI = 250

# Colors
COLOR_CYCLE = plt.rcParams['axes.prop_cycle'].by_key()['color']
MARKER_CYCLE = ['.', '*', 'x', 'o']
LS_CYCLE = ['-', '-.', ':', '--', (0, (1, 4))]

min_len = min(len(cycle) for cycle in (COLOR_CYCLE, LS_CYCLE))
cycler = mpl.cycler(color=COLOR_CYCLE[:min_len], ls=LS_CYCLE[:min_len])
# new_cycle = (mpl.cycler(color=COLOR_CYCLE) + mpl.cycler(linestyle=LS_CYCLE))

mpl.rcParams.update({
    'figure.dpi': 200,
    'font.size': FONTSIZE,
    'text.usetex': False,
    'legend.handlelength': 2,
    'legend.frameon': False,
    'axes.prop_cycle': cycler,
    'text.latex.preamble': (
        r'\usepackage{amsmath}'
        r'\usepackage{siunitx}'
        r'\usepackage{sansmath}'
        r'\sansmath'
        r'\sisetup{detect-all}'
        r'\newcommand{\avg}[1]{\underset{#1}{\mathrm{avg}} \, }'
    )
})

In [ ]:
KWARGS_NO_TICKS = {
    'bottom': False,
    'top': False,
    'left': False,
    'right': False,
    'labelbottom': False,
    'labeltop': False,
    'labelleft': False,
    'labelright': False
}

KWARGS_TOP_TICKS = {
    'bottom': False,
    'top': True,
    'left': False,
    'right': False,
    'labelbottom': False,
    'labeltop': True,
    'labelleft': False,
    'labelright': False
}

### Layouts

In [ ]:
from mpllayout import geometry as geo, layout as lay, matplotlibutils, solver, ui

In [ ]:
def gen_layout(
    fig_width,
    axes_shape,
    aspect_ratio=1,
    margins=None,
    thickness_colorbar=0.125
):
    """
    Create a layout of figures with fixed aspect ratio axes in a grid
    """
    if margins is None:
        margins = {
            'inner': 0.125,
            'left': 0.125,
            'right': 0.125,
            'top': 0.75,
            'bottom': 0.25
        }

    layout = lay.Layout()

    xmin, xmax = (0, 1)
    ymin, ymax = (0, 1)
    _verts = [[xmin, ymin], [xmax, ymin], [xmax, ymax], [xmin, ymax]]

    # Add the Figure
    box = geo.Box()
    layout.add_prim(geo.Quadrilateral(children=[geo.Point(vert) for vert in _verts]), 'Figure')
    layout.add_constraint(box, ('Figure',))

    # Fix the bottom left point of 'Figure' to the origin
    layout.add_constraint(
        geo.PointLocation(np.array([0.0, 0.0])), ('Figure/Line0/Point0',)
    )

    # Set the 'Figure' width
    layout.add_constraint(geo.Length(fig_width), ('Figure/Line0',))
    # layout.add_constraint(
    #     geo.Length(fig_height), ('Figure/Line1',)
    # )

    # Add all the Axes
    num_axes = int(np.prod(axes_shape))
    
    for n in range(num_axes):
        layout.add_prim(geo.Quadrilateral(children=[geo.Point(vert) for vert in _verts]), f'Axes{n}')
        layout.add_constraint(box, (f'Axes{n}',))

    # Constrain axes in a grid
    num_row, num_col = axes_shape
    xmargins = (num_col-1)*[margins['inner']]
    ymargins = (num_row-1)*[margins['inner']]
    rel_widths = (num_col-1)*[1]
    rel_heights = (num_row-1)*[1]
    axes_labels = tuple(f'Axes{n}' for n in range(num_axes))
    layout.add_constraint(geo.Grid(axes_shape, xmargins, ymargins, rel_widths, rel_heights), axes_labels)

    layout.add_constraint(geo.RelativeLength(aspect_ratio), ('Axes0/Line1', 'Axes0/Line0'))

    # Add a colorbar Axes
    
    layout.add_prim(geo.Quadrilateral(children=[geo.Point(vert) for vert in _verts]), 'AxesColorbar')
    layout.add_constraint(box, ('AxesColorbar',))

    # Force the colorbar to span the top axes
    collinear = geo.Collinear()
    layout.add_constraint(collinear, ('AxesColorbar/Line1', f'Axes{num_col-1}/Line1'))
    layout.add_constraint(collinear, ('AxesColorbar/Line3', 'Axes0/Line3'))
    layout.add_constraint(geo.YDistance(margins['inner']), ('Axes0/Line3/Point0', 'AxesColorbar/Line0/Point0'))
    layout.add_constraint(geo.Length(thickness_colorbar), ('AxesColorbar/Line3',))

    # Set the left + right margin
    layout.add_constraint(
        geo.XDistance(margins['left']), (f'Figure/Line0/Point0', f'Axes0/Line0/Point0')
    )
    layout.add_constraint(
        geo.XDistance(margins['right']), (f'Axes{num_col-1}/Line0/Point1', f'Figure/Line0/Point1')
    )

    # Set the top + bottom margin
    layout.add_constraint(
        geo.YDistance(margins['top']), (f'AxesColorbar/Line3/Point0', f'Figure/Line3/Point0')
    )
    layout.add_constraint(
        geo.YDistance(margins['bottom']), (f'Figure/Line0/Point0', f'Axes{num_axes-1}/Line0/Point0')
    )

    return solver.solve(layout.root_prim, layout.constraints, layout.constraint_graph, max_iter=25)

In [ ]:
prim_tree, solve_info = gen_layout(5, (3, 4), aspect_ratio=1)
print(solve_info)

fig, label_to_ax = matplotlibutils.subplots(prim_tree)

In [ ]:
fig

## Plot meshes (MPL and VTK)

In [ ]:
case_param = DEFAULT_CASE_PARAM.substitute(
    {
        'vcov': 1.07, 'psub': 400*10, 'dt': 5e-5, 'tf': 0.5,
        'SwellingDistribution': 'field.tavg_viscous_dissipation',
    }
)
print(case_param)
model = load_model(case_param)

mesh = model.solid.residual.mesh()
fspace_CG1 = dfn.VectorFunctionSpace(mesh, 'CG', 1)
xref = fspace_CG1.tabulate_dof_coordinates()[::3]

In [ ]:
## Load/specify a mesh displacement
with sf.StateFile(model, f'../out/{case_param.to_str()}.h5', mode='r') as f:
    u_mesh = f.get_state(0)['u']

### MPL

In [ ]:
# Plot mesh for Inkscape
fig, ax = plt.subplots(1, 1, figsize=(8.4*CM, 8.4*CM))

ax.set_aspect(1)

xy_vert = np.array(XREF)[VERT_TO_VDOF].reshape(-1, 2)
print(xy_vert.shape)
ax.triplot(*xy_vert.T, mesh.cells(), lw=0.5)

fig.tight_layout()
fig.savefig(f'{FIG_DIR}/Mesh.svg')

### VTK

In [ ]:
cells, cell_types, points = make_vtk_mesh_descr(model.solid.residual.mesh())

vert_to_vdof = dfn.vertex_to_dof_map(model.solid.residual.form['coeff.state.u1'].function_space())

In [ ]:
plotter = pv.Plotter(shape=(1, 1), border=True, window_size=(1200, 1200))

# grid = pv.UnstructuredGrid(cells, cell_types, points)
grid = pv.UnstructuredGrid(cells, cell_types, points)
grid.save('mesh.vtk')

grid = pv.UnstructuredGrid(cells, cell_types, points+2*u_mesh[vert_to_vdof].reshape(-1, 3))
grid.point_data['u'] = u_mesh[vert_to_vdof].reshape(-1, 3)
grid.save('mesh_swollen.vtk')

plotter.add_mesh(grid, show_edges=True, color=None)
# plotter.add_mesh(grid, show_edges=True, color=None, scalars=[p])

camera = plotter.camera
camera.focal_point = grid.center
camera.position = grid.center + 2.5*np.array((-0.25, 0.5, 1))
camera.up = (0, 1, 0)

# plotter.show()

img = plotter.screenshot()
fig, ax = plt.subplots(1, 1, figsize=(FIG_LX, FIG_LY))
ax.imshow(img)

## Tissue state evolution project

In [ ]:
DEFAULT_CASE_PARAM = DEFAULT_CASE_PARAM.substitute({
    'psub': 400*10, 'tf': 0.5, 'clscale': 0.75, 'NZ': 15
})

print(DEFAULT_CASE_PARAM)

model = load_model(DEFAULT_CASE_PARAM)

In [ ]:
fspace_vec_cg1 = model.solid.residual.form['coeff.state.u1'].function_space()
vert_to_dof_vec_cg1 = dfn.vertex_to_dof_map(fspace_vec_cg1)

fspace_dg0 = model.solid.residual.form['coeff.prop.v_swelling'].function_space()
# cell_to_dof_sca_dg0 = dfn.cell_to_dof_map(fspace_dg0)

# fspace_umesh = dfn.VectorFunctionSpace(model.solid.residual.mesh(), 'CG', 1)
vert_to_vdof = dfn.vertex_to_dof_map(fspace_vec_cg1)

### Animate VF dynamics

Plot the vocal fold self-sustained oscillations for a given case

In [ ]:
## Specify which damage/heal rate simulation to use
damage_measure = 'field.tavg_viscous_dissipation'

num_time_steps = 2
damage_rate = 1e-6
# damage_rate = 1e-9

# damage_measure = 'field.tmax_strain_energy'

# num_time_steps = 7
# damage_rate = 1e-4
# damage_rate = 1e-7

heal_rate = 1.3863e-1

vc_dir = f'../out/vicious_cycle'
vc_basename = f'DamageMeasure{damage_measure}--DamageRate{damage_rate:.4e}--HealRate{heal_rate:.4e}'

In [ ]:
with sf.StateFile(
    model, f'{vc_dir}/{vc_basename}--Step{num_time_steps}.h5', mode='r'
) as f:
    # idxs = np.arange(f.size-512, f.size)
    idxs = np.arange(0, 1024)
    ts = f.get_times()[idxs]
    us = [f.get_state(idx)['u'][vert_to_dof_vec_cg1].reshape(-1, 3) for idx in idxs]

In [ ]:
# INFO: Print maximum deformation component
np.max(np.abs(np.array(us)))

In [ ]:
SCALE = 1
N_PX_X = 800
N_PX_Y = int(round(13/13*N_PX_X))

## Setup the plotter with all visual elements
pl = pv.Plotter(
    shape=(1, 1),
    border=False,
    window_size=(N_PX_X, N_PX_Y)
)

grid = pv.UnstructuredGrid(cells, cell_types, np.array(points))
pl.add_mesh(grid, show_edges=True)

timer_text = pl.add_text('')

# scalar_bar_props = {
#     'position_x': 0.15,
#     'position_y': 0.005,
#     'label_font_size': 20,
#     'title_font_size': 20,
#     'title': "v",
#     'unconstrained_font_size': True,
#     'render': True,
#     'fmt': '%.4f',
#     'width': 0.7
# }
# pl.add_scalar_bar(**scalar_bar_props)

camera = pl.camera
camera.focal_point = grid.center
camera.position = grid.center + 3*np.array((-0.25, 0.25, 1))
camera.up = (0, 1, 0)

pl.open_movie('animation.mp4', framerate=60, quality=5)

## Write frames for each displacement time step
for u, t in zip(us, ts):
    grid.points = points + 1*u
    timer_text.set_text('upper_edge', f"t={1000*t:.0f} ms")
    pl.write_frame()

pl.close()

### Plot tissue state progression

In [ ]:
## Specify which damage/healing rate simulation to use
case_descr = {
    'damage_measure': 'field.tavg_viscous_dissipation',
    'num_time_steps': 6,
    'damage_rate': 1e-6
}

# case_descr = {
#     'damage_measure': 'field.tavg_viscous_dissipation',
#     'num_time_steps': 10,
#     'damage_rate': 1e-9
# }

# case_descr = {
#     'damage_measure': 'field.tavg_pos_strain_energy_rate',
#     'num_time_steps': 6,
#     'damage_rate': 1e-9
# }


# case_descr = {
#     'damage_measure': 'field.tavg_pos_strain_energy_rate',
#     'num_time_steps': 10,
#     'damage_rate': 1e-10
# }

case_descr['heal_rate'] = 1.3863e-1

vc_dir = f'../out/vicious_cycle'
vc_basename = (
    f"DamageMeasure{case_descr['damage_measure']}"
    f"--DamageRate{case_descr['damage_rate']:.4e}"
    f"--HealRate{case_descr['heal_rate']:.4e}"
)

In [ ]:
data = {}

def proc_prms(f: sf.StateFile):
    n = int(f.size//2)
    q = post.proc_glottal_flow_rate(f)
    prms = post.calc_prms(f.get_times()[n:], q[n:])
    return prms

def proc_q(f: sf.StateFile):
    n = int(f.size/2)
    q = f.file['state/fluid0.q'][:]
    # NOTE: `q` has shape (#voicing times, #planes)
    return q

def proc_qmid(f: sf.StateFile):
    q = proc_q(f)
    qmid = q[..., q.shape[-1]//2]
    return qmid

# Contains a function that post-processes the desired measurement from a
# given input simulation file
data_proc = {
    'qavg': post.proc_glottal_flow_rate,
    'qmid': proc_qmid,
    'q': proc_q,
    'vtime': lambda f: f.get_times(),
    'ttime': lambda f: f.file['ViciousCycle/time'][()],
    'damage_rate': lambda f: f.file['ViciousCycle/damage_rate'][:],
    'v': lambda f: f.get_prop()['v_swelling'][:],
    'u_swollen': lambda f: f.get_state(0)['u'],
    'stime': lambda f: f.get_times(),
    'psub': lambda f: f.get_control(0)['fluid0.psub'][0],
    'prms': proc_prms
}

for n in tqdm(range(case_descr['num_time_steps'])):
    with sf.StateFile(
        model, f'{vc_dir}/{vc_basename}--Step{n}.h5', mode='r'
    ) as f:
        for label, proc in data_proc.items():
            data_array = data.get(label, [])
            data_array.append(proc(f))
            data[label] = data_array

for key, array in data.items():
    data[key] = np.array(array)

# Compute closed ratio for 8th coronal plane (roughly halfway)
# `data['q']` has dimensions (# tissue time steps, # voicing time steps, # coronal planes)
# `data['qmid']` has dimensions (# tissue time steps, # voicing time steps)
n_trunc = data['qmid'].shape[0]//2
q_trunc = data['qmid'][:, n_trunc:]
t_trunc = data['vtime'][:, n_trunc:]

def closed_ratio(q, closed_ub):
    closed = np.count_nonzero(q < closed_ub)
    return closed/q.size

data['closed_ratio'] = np.array([
    closed_ratio(q, closed_ub=2*q.min()) 
    for q, t in zip(q_trunc, t_trunc)
])

# This doesn't work for some dumb reason
# data['closed_ratio'] = np.array([
#     clinical.closed_ratio(q, t, closed_ub=1.5*q.min()) 
#     for q, t in zip(q_trunc, t_trunc)
# ])

In [ ]:
mesh = model.solid.residual.mesh()
fspace = dfn.FunctionSpace(mesh, 'DG', 0)

dx = model.solid.residual.measure('dx')
_mf_label_to_value = model.solid.residual.mesh_function_label_to_value('cell')
measures = {
    key: dx(int(_mf_label_to_value[key])) for key in ('cover', 'body')
}
measures['total'] = dx

def integrate(f, dx, fspace):
    vfun = dfn.Function(fspace)
    vfun.vector()[:] = f
    return dfn.assemble(vfun*dx)

def volume_average(f, dx, fspace):
    # print(dx)
    return integrate(f, dx, fspace)/integrate(1, dx, fspace)

def function_probe(f, fspace, point):
    vfun = dfn.Function(fspace)
    vfun.vector()[:] = f
    # print(dx)
    return vfun(point)

# DEBUG: Just a quick check
# print(function_probe(data['v'][-1], fspace, (0.2, 0.2, 0.75)))

data_proc = {
    measure_name:  lambda vs, dx: np.array([volume_average(v, dx, fspace) for v in vs])
    for measure_name in measures.keys()
}
_data = {
    f'v_{measure_name}_avg': proc(data['v'], measures[measure_name]) 
    for measure_name, proc in data_proc.items()
}
data.update(_data)

V_PROBE_POINTS = [(0.65, 0.48, 0.75), (0.65, 0.2, 0.75)]
data_proc = {
    point: lambda vs, point: np.array([function_probe(v, fspace, point) for v in vs])
    for point in V_PROBE_POINTS
}
_data = {
    f'v_at_{point}': proc(data['v'], point) 
    for point, proc in data_proc.items()
}
data.update(_data)

def indicator_lb(v, lb):
    clip_v = np.ones(v.shape)
    clip_v[v < lb] = 0
    return clip_v

lbs = (5, 10, 15)
dx = measures['total']
_data = {
    f'percent_volume_over_{lb:d}': np.array(
        [integrate(indicator_lb(v, 1+lb/100), dx, fspace)/integrate(1, dx, fspace) for v in data['v']]
    )
    for lb in lbs
}
data.update(_data)
data['v_total_max'] = np.array([v.max() for v in data['v']])

In [ ]:
for point in V_PROBE_POINTS:
    print(data[f'v_at_{point}'])

#### Plot compensatory subglottal pressure and volume

In [ ]:
def plot_psub_volume_trend(fig, axs, time, psub, vtotal, idx=None, clip=True):
    
    axs[0].plot(time, psub)
    axs[0].set_ylabel(r"$p_\mathrm{sub}$ $[\mathrm{Pa}]$")

    axs[1].plot(time, 100*(vtotal-1))
    axs[1].set_ylabel(r"$\Delta V/V_0$ $[\%]$")

    axs[-1].set_xticks(time)
    axs[-1].set_xlabel("Time $[\mathrm{h}]$")
    # axs[-1].set_xlim(np.round(np.min(vc_ts)), np.round(np.max(vc_ts)))

In [ ]:
if FIG_TYPE == 'manuscript':
    figsize=(FIG_LX, FIG_LX)
elif FIG_TYPE == 'presentation':
    figsize=(FIG_LX_WIDE/2, FIG_LY)

fig, axs = plt.subplots(2, 1, figsize=figsize, sharex=True)

axs[0].plot(data['ttime'], data['psub']/10)
axs[0].set_ylabel(r"$p_\mathrm{sub}$ $[\mathrm{Pa}]$")

for key in ('total', 'cover', 'body'):
    axs[1].plot(data['ttime'], 100*(data[f'v_{key}_avg']-1), label=key)
axs[1].set_ylabel(r'Average $\Delta{v} \; [\%]$')
axs[1].legend()

axs[-1].set_xticks(data['ttime'])
axs[-1].set_xlabel("Time $[\mathrm{h}]$")

fig.tight_layout()

fig.savefig(f"{FIG_DIR}/PsubVolumeTrend--{case_descr['damage_measure']}--{case_descr['damage_rate']:.2e}.{FIG_EXT}")

#### Plot acoustic and vibration features

In [ ]:
if FIG_TYPE == 'manuscript':
    figsize=(FIG_LX, FIG_LX)
elif FIG_TYPE == 'presentation':
    figsize=(FIG_LX_WIDE/2, FIG_LY)

fig, axs = plt.subplots(2, 1, figsize=figsize)

idxs = [0, 3, len(data['ttime'])-1]
for ttime, vtime, q in zip(data['ttime'][idxs], data['vtime'][idxs], data['qmid'][idxs]):
    axs[0].plot(vtime, q, label=f"{ttime:.1f} h")
axs[0].set_ylabel("$q$ $[\mathrm{cm}^3\mathrm{s}^{-1}]$")
axs[0].set_xlabel("Voicing time $[\mathrm{s}]$")
axs[0].set_xlim(0.3, 0.31)
axs[0].legend()

axs[1].plot(data['ttime'], 100*data['closed_ratio'])
axs[1].plot(data['ttime'][idxs], 100*data['closed_ratio'][idxs], ls='', marker='o', mfc='none', mec='k', ms=5)
axs[1].set_ylabel("$\mathrm{CQ}$ [%]")

axs[1].set_xticks(data['ttime'])
axs[1].set_xlabel("Tissue time $[\mathrm{h}]$")

fig.tight_layout()

fig.savefig(f"{FIG_DIR}/GlottalFlowWaveformTrend--{case_descr['damage_measure']}--{case_descr['damage_rate']:.2e}.{FIG_EXT}")

#### Plot detailed volume progressions

In [ ]:
if FIG_TYPE == 'manuscript':
    figsize=(FIG_LX, FIG_LX)
elif FIG_TYPE == 'presentation':
    figsize=(FIG_LX_WIDE/2, FIG_LY)

fig, axs = plt.subplots(2, 1, figsize=figsize, sharex=True)

print(data[f'v_at_{point}'])
for point in V_PROBE_POINTS:
    axs[0].plot(data['ttime'], 100*(data[f'v_at_{point}']-1), label=f'{point}')
    axs[0].set_ylabel(r"$\Delta{v}(\mathbf{x})$ $[\%]$")
    axs[0].legend()


for lb in (5, 10, 15):
    axs[1].plot(data['ttime'], 100*data[f'percent_volume_over_{lb:d}'], label=r"$\Delta{v}_\mathrm{lb}$ " f"{lb:d}%")
axs[1].set_ylabel(r"$\Delta{v} > \Delta{v}_\mathrm{lb}$ $\,[\%]$")
axs[1].legend()

fig.tight_layout()

fig.savefig(f"{FIG_DIR}/DetailedVolumeTrend--{case_descr['damage_measure']}--{case_descr['damage_rate']:.2e}.{FIG_EXT}")

#### Plot tissue state progression

In [ ]:
mesh = model.solid.residual.mesh()
cells, cell_types, points = make_vtk_mesh_descr(mesh)

In [ ]:
## Specify the time steps and coronal plane locations to plot

time_idxs = [0, 1, 2, 3, 4]
# time_idxs = range(num_time_steps)
time_idxs = [0, int(case_descr['num_time_steps']/2), case_descr['num_time_steps']-1]

# NOTE: These are z-coordinates of coronal planes. This will change with the mesh.
z_planes = np.linspace(0, 1.5, 15)
# z_clips = [z_planes[z_planes.size//2], z_planes[z_planes.size//4], z_planes[z_planes.size//6]]
z_clips = 0.75*np.array([1, 1-1/3, 1-2/3])

# z_clips = 0.75*np.array([1-1/3])
z_clips

In [ ]:
## Create the axes layout
margins = {
    'top': 1/2,
    'bottom': 1/4,
    'left': 3/8,
    'right': 1/8,
    'inner': 1/16
}

axes_shape = (len(time_idxs), len(z_clips))
prim_tree, solve_info = gen_layout(FIG_LX_WIDE, axes_shape, aspect_ratio=1, margins=margins)
print(solve_info)

In [ ]:
fig, label_to_ax = matplotlibutils.subplots(prim_tree)

axs = np.array([label_to_ax[f'Axes{n}'] for n in range(np.prod(axes_shape))])
axs = axs.reshape(axes_shape)
ax_cbar = label_to_ax['AxesColorbar']

fig.frameon = False

In [ ]:
def format_camera(camera: pv.Camera, _grid: pv.Grid, clipped_grid: pv.Grid):
    """
    Return a formatted `pyvista.Camera` standard view
    """
    # This is useful for perspective projection
    # camera.position = grid.center + 2.0*np.array((-0.25, 0.5, 1))
    # camera.position = (*xy, z_clip) + 2.0*np.array((0.0, 0.0, 1))
    
    # This is useful for parallel projection
    xy = grid.center[:2]
    camera.focal_point = (*xy, 0)
    camera.position = (*xy, z_clip+1)
    # camera.position = (0, 0, z_clip+2)
    camera.up = (0, 1, 0)
    camera.parallel_projection = True
    # NOTE: I set this scale manually!
    camera.parallel_scale = 0.45
    return camera

In [ ]:
for ax in axs.flat:
    ax.tick_params(axis='both', which='both', **KWARGS_NO_TICKS)

v_min = 1.0
v_max = np.max([np.max(v) for v in [data['v'][n] for n in time_idxs]])
# v_max = 1.1

from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
fig.colorbar(
    ScalarMappable(cmap='viridis', norm=Normalize(vmin=v_min, vmax=v_max)),
    cax=ax_cbar, orientation='horizontal',
)
ax_cbar.xaxis.set_tick_params(**KWARGS_TOP_TICKS)
ax_cbar.set_xlabel("$v$ ")
ax_cbar.xaxis.set_label_position('top')

for ax, (time_idx, z_clip) in zip(axs.flat, itertools.product(time_idxs, z_clips)):
    deformation_scale = 2
    points_swollen = points + deformation_scale*data['u_swollen'][time_idx, vert_to_vdof].reshape(-1, 3)
    v_swell = data['v'][time_idx]
    grid = pv.UnstructuredGrid(cells, cell_types, points)
    grid_swollen = pv.UnstructuredGrid(cells, cell_types, points_swollen)

    # Export an image from a pyvista plot of the mesh
    pl = pv.Plotter(shape=(1, 1), border=False, window_size=(1200, 1200))

    pl.add_points(np.array(V_PROBE_POINTS), style='points_gaussian', point_size=20)
    # pl.show_bounds()
    plot_cell_scalar_kwargs = {
        'clip_origin': (0, 0, z_clip), 'clim': (v_min, v_max), 'clip_normal': (0, 0, 1)
    }
    img = plot_cell_scalar(
        pl, grid_swollen, v_swell, format_camera, **plot_cell_scalar_kwargs
    )
    pl.close()
    
    ax.imshow(img)

for ax, time_idx in zip(axs[:, 0], time_idxs):
    vc_time = data['ttime'][time_idx]
    ax.set_ylabel(f"t={vc_time:.2f} h")

for ax, z_clip in zip(axs[-1, :], z_clips):
    ax.set_xlabel(f"z={z_clip:.2f} cm")

fig.savefig(f"{FIG_DIR}/SwellingProgression--{case_descr['damage_measure']}--{case_descr['damage_rate']:.2e}.{FIG_EXT}")

fig

#### Plot tissue state progression successively

In [ ]:
def pvplot_fancy(pl, u_swell, v_swell, vc_time, n_vc):
    _points = POINTS+SCALE*u_swell[vert_to_dof_vec_cg1].reshape(-1, 3)
    grid = pv.UnstructuredGrid(cells, cell_types, _points)
    # print(np.min(v_swell), np.max(v_swell))
    grid.cell_data['v'] = v_swell
    clipped_grid = grid.clip(normal=(0, 0, 1), origin=(0, 0, 0.75), )

    pl.subplot(0, 0)
    # pl.add_mesh(grid, scalars='v')
    pl.add_mesh(grid, show_scalar_bar=False, style='wireframe', opacity=0.05, edge_color='black')
    _style = {
        'cmap': 'viridis',
        'show_edges': True,
        'show_scalar_bar': False,
        'edge_color': 'white'
    }
    pl.add_mesh(clipped_grid, scalars='v', clim=(1, v_max), **_style)

    pl.add_text(f"Time: {vc_time:.1f} H", position='upper_edge', font_size=20)

    ## Plot swelling state
    pl.subplot(0, 0)
    # pl.link_views()
    scalar_bar_props = {
        'position_x': 0.15,
        'position_y': 0.005,
        'label_font_size': 18,
        'title_font_size': 18,
        'title': "v",
        'unconstrained_font_size': True,
        'render': True,
        'fmt': '%.4f',
        'width': 0.7

    }
    pl.add_scalar_bar(**scalar_bar_props)

    center = pv.UnstructuredGrid(cells, cell_types, POINTS).center
    camera = pl.camera
    camera.focal_point = center
    camera.position = center + 3*np.array((-0.25, 0.5, 1))
    camera.up = (0, 1, 0)
    # camera.zoom('tight')

    ## Plot chart with compensatory subglottal pressure
    pl.subplot(0, 1)

    fig, axs = plt.subplots(2, 1, figsize=(12*CM, 12*CM), sharex=True)
    plot_psub_volume_trend(fig, axs, n_vc)
    fig.tight_layout()

    pl.add_chart(pv.ChartMPL(fig))

In [ ]:
time_idxs = [0, 3, 6, 9]
SCALE = 1000

# time_idxs = [0, 1, 2]
# time_idxs = [0, 3, 6]
# SCALE = 2
DPCM = 50

N_PX_X = 30 * DPCM
N_PX_Y = 13 * DPCM

In [ ]:
for ii, n_vc in enumerate(time_idxs):
    pl = pv.Plotter(
        shape=(1, 2),
        border=False,
        window_size=(N_PX_X, N_PX_Y), border_width=0.0
    )

    # us_swell = [0]
    v_max = np.max(np.array([vfields[n] for n in time_idxs]))

    # n_vc = 0
    u_swell = us_swell[n_vc]
    v_swell = vfields[n_vc]

    pvplot_fancy(pl, u_swell, v_swell, vc_ts[n_vc], n_vc)

    pl.show()

    fig_name = f'FancyVCProgression--{vc_basename}--{ii}'
    # pl.save_graphic(f'{FIG_DIR}/{fig_name}.{FIG_EXT}')
    pl.screenshot(f'{FIG_DIR}/{fig_name}.png')

    pl.close()

### Linearized tissue state analysis

In [ ]:
## Load the main results
post_dir = '../out'

DATA = {}
with h5py.File(f'{post_dir}/postprocess.h5', mode='r') as f:
    DATA = h5utils.h5_to_dict(f, DATA)

In [ ]:
acoustic_data = postproc_acoustics_from_data(DATA)

DATA.update(acoustic_data)

In [ ]:
mesh = model.solid.residual.mesh()
fspace = dfn.FunctionSpace(mesh, 'DG', 0)
dx = dfn.Measure('dx', mesh)

fieldavg_data = postproc_fieldavg_from_data(DATA, fspace, dx, 'field.tavg_viscous_dissipation')
DATA.update(fieldavg_data)

fieldavg_data = postproc_fieldavg_from_data(DATA, fspace, dx, 'field.tavg_pos_strain_energy_rate')
DATA.update(fieldavg_data)

In [ ]:
casenames = get_casenames(DATA)
resultnames = get_resultnames(DATA)

# print(casenames)
# print(resultnames)

#### Plot unswollen damage distributions

In [ ]:
_param = {
    'psub': 4e3, 'clscale': 0.75, 'NZ': 15, 'SwellingDistribution': 'field.tavg_pos_strain_energy_rate',
    'mcov': 0.0, 'tf': 0.5
}
case_param = DEFAULT_CASE_PARAM.substitute(_param)

In [ ]:
damage_measure_names = ['field.tavg_pos_strain_energy_rate', 'field.tavg_viscous_dissipation']
damage_measure_display_names = [r'$< \dot{w} >$', r'$\dot{w}_\mathrm{viscous}$']

damage_fields = {
    field_name: load_dataset(DATA, field_name, case_param)
    for field_name in damage_measure_names
}
damage_field_display_names = {
    name: display_name for name, display_name in zip(damage_measure_names, damage_measure_display_names)
}

In [ ]:
model = load_model(case_param)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(FIG_LX, FIG_LY*0.4))

for ax in axs.flat:
    # ax.tick_params(axis='both', which='both', **KWARGS_NO_TICKS)
    ax.set_axis_off()

for ax, damage_field_name in zip(axs, damage_fields.keys()):
    damage_field = damage_fields[damage_field_name]
    damage_field_display_name = damage_field_display_names[damage_field_name]
    
    plotter = pv.Plotter(shape=(1, 1), border=True, window_size=(400, 400))
    grid = pv.UnstructuredGrid(*make_vtk_mesh_descr(model.solid.residual.mesh()))
    def format_camera(camera, grid, clipped_grid):
        camera.focal_point = grid.center
        camera.position = grid.center + 1.5*np.array((-0.25, 0.5, 1))
        camera.up = (0, 1, 0)
    img = plot_cell_scalar(plotter, grid, damage_field, format_camera)

    ax.imshow(img)
    text_kwargs = {'transform': ax.transAxes, 'ha': 'left', 'va': 'top'}
    ax.text(0.05, 0.95, damage_field_display_name, **text_kwargs)

fig.savefig(f'{FIG_DIR}/UnswollenDamageDistributions.{FIG_EXT}')

#### Plot summary of parametric variations vs vcov, psub etc

In [ ]:
mcov = 0.0

swelling_distribution = 'field.tavg_viscous_dissipation'
swelling_distribution = 'field.tavg_pos_strain_energy_rate'

# _param = {
#     'psub': 4e3, 'clscale': 9.40e-1, 'NZ': 12, 'SwellingDistribution': swelling_distribution,
#     'mcov': 0.0, 'tf': 5e-5
# }
_param = {
    'psub': 4e3, 'clscale': 0.75, 'NZ': 15, 'SwellingDistribution': swelling_distribution,
    'mcov': mcov, 'tf': 0.5
}
default_param = DEFAULT_CASE_PARAM.substitute(_param)
print(default_param.to_str())

vcovs = np.array([1.0, 1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07])
# vcovs = np.array([1.0])
case_params_vcov = [default_param.substitute({'vcov': vcov}) for vcov in vcovs]

psubs = 10*np.array([400, 410])
# psubs = 10*np.array([400])
case_params_psub = [default_param.substitute({'psub': psub, 'vcov': 1}) for psub in psubs]

# damage_measure = f'cover_avg_{swelling_distribution}'
damage_measure = f'cover_avg_{swelling_distribution}'
damage_vs = {}
damage_vs['psub'] = np.array(load_datasets(DATA, damage_measure, case_params_psub))
damage_vs['vcov'] = np.array(load_datasets(DATA, damage_measure, case_params_vcov))

spl_vs = {}
spl_vs['vcov'] = np.array(load_datasets(DATA, 'spl', case_params_vcov))
spl_vs['psub'] = np.array(load_datasets(DATA, 'spl', case_params_psub))

In [ ]:
model = load_model(default_param)

In [ ]:
if FIG_TYPE == 'manuscript':
    fig, axs = plt.subplots(2, 2, figsize=(FIG_LX_WIDE, 0.5*FIG_LY))
elif FIG_TYPE == 'presentation':
    fig, axs = plt.subplots(2, 2, figsize=(FIG_LX_WIDE, FIG_LY))

xs = (vcovs, psubs/10)
x_labels = ('vcov', 'psub')
for ax, x, x_label in zip(axs[0, :], xs, x_labels):
    ax.plot(x, damage_vs[x_label])

for ax, x, x_label in zip(axs[1, :], xs, x_labels):
    ax.plot(x, spl_vs[x_label])

for axs_row in axs:
    for ax in axs_row[1:]:
        ax.sharey(axs_row[0])

for axs_col in axs.T:
    for ax in axs_col[1:]:
        ax.sharex(axs_col[0])

x_labels = ('$v_\mathrm{cov} \, [\,]$', '$p_\mathrm{sub} \, [\mathrm{Pa}]$')
for ax, x_label in zip(axs[-1, :], x_labels):
    ax.set_xlabel(x_label)

y_labels = (
    'Spatial avg. $\dot{w} \, [10^{-7} \mathrm{W}]$',
    '$\mathrm{SPL} \, [\mathrm{dB}]$'
)
for ax, y_label in zip(axs[:, 0], y_labels):
    ax.set_ylabel(y_label)

format_axs_grid(axs)

# fig.tight_layout()
fig.savefig(f'{FIG_DIR}/ViciousCycleFactorSummary--{swelling_distribution}--{mcov:.2e}.{FIG_EXT}')

#### Plot swollen geometries vs. $v$

In [ ]:
vcovs = np.array([1.0, 1.01, 1.02, 1.03, 1.04])
mcovs = [-0.0]

pdefault = DEFAULT_CASE_PARAM

In [ ]:
# Load the first state/displacement for each swelling case
# ; this is the swelling displacement because we solved for the
# static equilibrium before integrating in time
mcov = -0.8
case_param = [
    DEFAULT_CASE_PARAM.substitute({'vcov': vcov, 'mcov': mcov}) for vcov, mcov in itertools.product(vcovs, mcovs)
]

swelling_displacements = {}
for param in case_param:
    with sf.StateFile(model, f'{OUT_DIR}/{param.to_str()}.h5') as f:
        swelling_displacements[param['vcov']] = f.get_state(0).sub['u']

In [ ]:
if FIG_TYPE == 'manuscript':
    fig, ax = plt.subplots(1, 1, figsize=SIZE_SMALL, sharex=True)
elif FIG_TYPE == 'presentation':
    fig, ax = plt.subplots(1, 1, figsize=(FIG_LX_WIDE/2, FIG_LY), sharex=True)
# fig, ax = plt.subplots(1, 1, figsize=SIZE_SMALL)

scale = 2
vcovs = np.array([1.0, 1.3])
for vcov, color, marker, ls in zip(vcovs, COLOR_CYCLE, MARKER_CYCLE, LS_CYCLE):
    u_dof = swelling_displacements[vcov]
    xcur_dof = XREF + scale*u_dof
    xcur_ver = xcur_dof[VERT_TO_VDOF].reshape(-1, 2)

    # This plots the whole triangulation/mesh
    # triang = tri.Triangulation(xcur_ver[:, 0], xcur_ver[:, 1], triangles=CELLS)
    # ax.triplot(triang, color=color)

    # Plot the medial and interface surfaces
    vert_med = np.concatenate([VERT_MED, VERT_MED[:1]])
    kwargs_line_style = {
        'color': color,
        'ls': ls
        # 'marker': marker,
        # 'markevery': 5
    }
    ax.plot(*xcur_ver[vert_med].T, **kwargs_line_style)
    ax.plot(*xcur_ver[VERT_INT].T, **kwargs_line_style)

# Create a custom legend for each swelling level
from matplotlib.lines import Line2D
lines = [
    Line2D([0], [0], color=color, ls=ls)
    for color, ls in zip(COLOR_CYCLE[:len(vcovs)], LS_CYCLE)
]
labels = [f"$v$={vcov:.2f}" for vcov in vcovs]
ax.legend(lines, labels)

ax.set_aspect(1)
ax.set_xlabel("x $[\mathrm{cm}]$")
ax.set_ylabel("y $[\mathrm{cm}]$")
fig.tight_layout()
fig.savefig(f"{FIG_DIR}/SwollenGeometryVsV.{FIG_EXT}", dpi=DPI)

In [ ]:
## Plot for P50 presentation
vs = [1., 1.3]

for v in vs:
    u_dof = swelling_displacements[v]
    xcur_dof = XREF + scale*u_dof
    xcur_ver = xcur_dof[VERT_TO_VDOF].reshape(-1, 2)

    fig, ax = plt.subplots(1, 1, figsize=SIZE_SMALL)
    ax.triplot(*xcur_ver.T, CELLS, lw=0.75)
    ax.triplot(*xcur_ver.T, CELLS[5:6], lw=1, color='r')
    ax.set_aspect(1)
    ax.yaxis.set_tick_params(label1On=False, tick1On=False)
    ax.xaxis.set_tick_params(label1On=False, tick1On=False)

    fig.savefig(f'{FIG_DIR}/SwellingV{v:.1f}.{FIG_EXT}')

#### Plot mass and volume vs. $v$

In [ ]:
from matplotlib.lines import Line2D

vcovs = np.array([1.0, 1.025, 1.05, 1.075, 1.1])
vcovs = np.array([1.0, 1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07])
mcov = -0.8

swelling_distribution = 'field.tavg_viscous_rate'
# swelling_distribution = 'field.tmax_strain_energy'

pdefault = DEFAULT_CASE_PARAM.substitute({'psub': 400*10, 'SwellingDistribution': swelling_distribution})

In [ ]:
case_params = [pdefault.substitute({'vcov': vcov, 'mcov': mcov, 'tf': 0.5}) for vcov in vcovs]
masses_cov = np.array(load_datasets(DATA, 'mass_cover', case_params))
masses_tot = np.array(load_datasets(DATA, 'mass_total', case_params))

vols_cov = np.array(load_datasets(DATA, 'vol_cover', case_params))
vols_tot = np.array(load_datasets(DATA, 'vol_total', case_params))

vols_cov = np.array(load_datasets(DATA, 'vol_cover', case_params))

In [ ]:
masses_cov
vols_cov

In [ ]:
## Plot showing cover volume vs swelling
lss = ['-',  '-.', ':']
fig, ax = plt.subplots(1, 1, sharex=True, figsize=(FIG_LX_WIDE/2, FIG_LY))
ax.plot(vcovs, vols_cov, ls=lss[0], label="absolute cover volume", color=COLOR_CYCLE[0])
ax_twin = ax.twinx()
ax_twin.plot(vcovs, 100*vols_cov/vols_tot, ls=lss[1], color=COLOR_CYCLE[1], label="relative to total VF volume")
ax_twin.plot(vcovs, 100*(vols_cov-vols_cov[0])/vols_cov[0], ls=lss[2], color=COLOR_CYCLE[1], label="relative to initial cover volume")
ax_twin.set_ylabel("Cover volume $[\si{\percent}]$")
ax_twin.set_ylim(0, 30)

# ax_twin.legend(loc=(0, 1.05))
# ax.legend(loc=())

colors = [COLOR_CYCLE[0]] + [COLOR_CYCLE[1]]*2

lines = [Line2D([], [], color=color, ls=ls) for color, ls in zip(colors, lss)]
labels = [
    'absolute (left axis) volume $[\si{\cm^2}]$ ',
    'relative (right axis) \n to total VF volume $[\si{\percent}]$ ',
    'relative (right axis) \n to initial cover volume $[\si{\percent}] $'
]
# ax.legend(lines, labels, loc=(0, 1.05), handlelength=3)

ax.legend(lines, labels, loc='lower left', bbox_to_anchor=(-0.075, 1.05, 1.15, 0.3), mode='expand')

ax.set_ylabel("Cover volume $[\si{\cm^2}]$")
ax.set_xlabel("$v$")

fig.tight_layout()
fig.savefig(f"{FIG_DIR}/CoverVolumeVsV--{swelling_distribution}.{FIG_EXT}")

In [ ]:
## Plot showing cover volume + mass vs swelling
fig, axs = plt.subplots(2, 1, sharex=True, figsize=(FIG_LX_WIDE/2, FIG_LY))
axs[0].plot(vcovs, masses_cov)
ax_twin = axs[0].twinx()
ax_twin.plot(vcovs, 100*masses_cov/masses_tot, ls=':')
ax_twin.set_ylabel("% total")

axs[1].plot(vcovs, vols_cov)
ax_twin = axs[1].twinx()
ax_twin.plot(vcovs, 100*vols_cov/vols_tot, ls=':')
ax_twin.set_ylabel("% total")

axs[0].set_ylabel("Cover mass [\si{\g}]")
axs[1].set_ylabel("Cover volume [\si{\cm^2}]")

axs[1].set_xlabel("$v$ [1]")

fig.tight_layout()
# fig.savefig(f"{FIG_DIR}/CoverMassVsV.png")
# fig.savefig(f"{FIG_DIR}/CoverMassVsV.pdf")

#### Plot (SPL, $f_\mathrm{o}$, Amplitude) vs. $v$

In [ ]:
vcovs = np.array([1.0, 1.025, 1.05, 1.075, 1.1])
vcovs = np.array([1.0, 1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07])
mcovs = np.array([0.0, -0.8])

swelling_distribution = 'field.tavg_viscous_rate'
# swelling_distribution = 'field.tmax_strain_energy'

DEFAULT_CASE_PARAM = DEFAULT_CASE_PARAM.substitute(
    {
        'psub': 400*10, 'ModifyEffect': '', 'tf': 0.5, 'SwellingDistribution': swelling_distribution
    }
)

In [ ]:
params = [default_param.substitute({'vcov': vcov}) for vcov in vcovs]
prms_trends = {
    mcov: load_datasets(
        DATA, 'prms', params, {'mcov': mcov}
    )
    for mcov in mcovs
}
data_f0 = {
    mcov: load_datasets(
        DATA, 'fund_freq', params, {'mcov': mcov}
    )
    for mcov in mcovs
}
data_df0 = {
    mcov: load_datasets(
        DATA, 'dfreq', params, {'mcov': mcov}
    )
    for mcov in mcovs
}
data_amp = {
    mcov: load_datasets(
        DATA, 'amplitude', params, {'mcov': mcov}
    )
    for mcov in mcovs
}

In [ ]:
if FIG_TYPE == 'manuscript':
    fig, axs = plt.subplots(3, 1, figsize=SIZE_MED, sharex=True)
elif FIG_TYPE == 'presentation':
    fig, axs = plt.subplots(1, 2, figsize=(FIG_LX_WIDE, FIG_LY), sharex=True)

for mcov, color, ls in zip(mcovs, COLOR_CYCLE, LS_CYCLE):
    y = data_f0[mcov]
    dy = data_df0[mcov]
    line_kwargs = {'ls': ls, 'color': color}

    axs[0].plot(vcovs, y, label=f"$\\bar{{m}}'={mcov:.1f}$", **line_kwargs)
    print(y)

    prmss = prms_trends[mcov]
    spls = 20*np.log10(prmss/20e-6)
    axs[1].plot(vcovs, spls, label=f"$\\bar{{m}}'={mcov:.1f}$", **line_kwargs)

    if FIG_TYPE != 'presentation':
        amps = data_amp[mcov]
        axs[2].plot(vcovs, amps, label=f"$\\bar{{m}}'={mcov:.1f}$", **line_kwargs)

axs[0].legend(loc='lower left', ncol=3, bbox_to_anchor=(0, 1.0, 1.0, 0.1), mode='expand')
axs[0].set_ylabel("$f_o$ $[\si{\Hz}]$")
axs[1].set_ylabel("$\mathrm{SPL}$ $[\si{\dB}]$")

if FIG_TYPE != 'presentation':
    axs[2].set_ylabel("Amplitude $[\si{\cm}]$")

n = -3
mcov = 0.0
v = vcovs[n]
f0s = [data_f0[mcov][n] for mcov in mcovs[[0, -1]]]
spls = [20*np.log10(prms_trends[mcov]/20e-6)[n] for mcov in mcovs[[0, -1]]]
annotate_vertical_trend(axs[0], "increasing softening", v, *f0s)
annotate_vertical_trend(axs[1], "increasing softening", v, *spls)


if FIG_TYPE == 'manuscript':
    axs[-1].set_xlabel("$v$")
elif FIG_TYPE == 'presentation':
    for ax in axs:
        ax.set_xlabel("$v$")
        ax.set_xticks(vcovs)

fig.tight_layout()
fig.savefig(f"{FIG_DIR}/F0andSPLVsV--{swelling_distribution}.{FIG_EXT}")

#### Summarize $f_\mathrm{o}$ and phonotrauma measure

In [ ]:
vcovs = np.array([1.0, 1.025, 1.05, 1.075, 1.1])
vcovs = np.array([1.0, 1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07])
mcovs = np.array([0.0, -0.8])

swelling_distribution = 'field.tavg_viscous_rate'
# swelling_distribution = 'field.tmax_strain_energy'
default_param = DEFAULT_CASE_PARAM.substitute(
    {
        'psub': 400*10, 'ModifyEffect': '', 'tf': 0.5, 'SwellingDistribution': swelling_distribution
    }
)

In [ ]:
params = [default_param.substitute({'vcov': vcov}) for vcov in vcovs]
dat_f0 = {
    mcov: load_datasets(
        DATA, 'fund_freq', params, {'mcov': mcov}
    )
    for mcov in mcovs
}

dat_spl = {
    mcov: load_datasets(
        DATA, 'spl', params, {'mcov': mcov}
    )
    for mcov in mcovs
}

dat_wvisc = {
    mcov: load_datasets(
        DATA, 'time.spatial_stats_viscous', params, {'mcov': mcov}
    )
    for mcov in mcovs
}

In [ ]:
fig = plt.figure(
    figsize=(FIG_LX_WIDE, FIG_LY), constrained_layout=True
)

gs = mpl.gridspec.GridSpec(1, 2, figure=fig)
ax_acous = fig.add_subplot(gs[0])
ax_wvisc = fig.add_subplot(gs[1])
# ax_acous.xaxis.set_tick_params(labelbottom=False)

for mcov, color, ls in zip(mcovs, COLOR_CYCLE, LS_CYCLE):
    kwargs = {
        'color': color, 'ls': ls,
        'label': f"$\\bar{{m}}'={mcov:.1f}$"
    }
    ax_acous.plot(vcovs, dat_f0[mcov], **kwargs)

    N = dat_wvisc[mcov].shape[-1]
    ys = np.mean(dat_wvisc[mcov]['avg'][..., -N//2:], axis=-1)
    ax_wvisc.plot(vcovs, ys*1e-7/(1e-2**2), **kwargs)

    ax_acous.legend()

n = 3
v = vcovs[n]
wviscs = [
    1e-7/(1e-2**2)*np.mean(dat_wvisc[mcov]['avg'][..., -N//2:][n], axis=-1)
    for mcov in mcovs[[0, -1]]
]
f0s = [dat_f0[mcov][n] for mcov in mcovs[[0, -1]]]
annotate_vertical_trend(ax_acous, "increasing softening", v, *f0s)
annotate_vertical_trend(ax_wvisc, "increasing softening", v, *wviscs)

ax_acous.set_ylabel("$f_o$ [Hz]")
ax_wvisc.set_ylabel(r"$\hat{w}_\mathrm{visc}$ [\si{\W\m^{-2}}]")

for ax in (ax_acous, ax_wvisc):
    ax.set_xlabel("$v$")

fig.savefig(f'{FIG_DIR}/FundamentalFrequencyandPhonotraumaSummary--{swelling_distribution}.{FIG_EXT}')

In [ ]:
fig = plt.figure(
    figsize=(FIG_LX_WIDE, FIG_LY), constrained_layout=True
)

gs = mpl.gridspec.GridSpec(1, 2, figure=fig)
ax_acous = fig.add_subplot(gs[0])
ax_wvisc = fig.add_subplot(gs[1])
# ax_acous.xaxis.set_tick_params(labelbottom=False)

for mcov, color, ls in zip(mcovs, COLOR_CYCLE, LS_CYCLE):
    kwargs = {
        'color': color, 'ls': ls,
        'label': f"$\\bar{{m}}'={mcov:.1f}$"
    }
    ax_acous.plot(vcovs, dat_spl[mcov], **kwargs)

    N = dat_wvisc[mcov].shape[-1]
    ys = np.mean(dat_wvisc[mcov]['avg'][..., -N//2:], axis=-1)
    ax_wvisc.plot(vcovs, ys*1e-7/(1e-2**2), **kwargs)

    ax_acous.legend()

n = 3
v = vcovs[n]
wviscs = [
    1e-7/(1e-2**2)*np.mean(dat_wvisc[mcov]['avg'][..., -N//2:][n], axis=-1)
    for mcov in mcovs[[0, -1]]
]
f0s = [dat_spl[mcov][n] for mcov in mcovs[[0, -1]]]
annotate_vertical_trend(ax_acous, "increasing softening", v, *f0s)
annotate_vertical_trend(ax_wvisc, "increasing softening", v, *wviscs)

ax_acous.set_ylabel("$\mathrm{SPL}$ [Hz]")
ax_wvisc.set_ylabel(r"$\hat{w}_\mathrm{visc}$ [\si{\W\m^{-2}}]")

for ax in (ax_acous, ax_wvisc):
    ax.set_xlabel("$v$")

fig.savefig(f'{FIG_DIR}/SPLandPhonotraumaSummary--{swelling_distribution}.{FIG_EXT}')

### Postprocess linearized tissue state information

In [ ]:
vcovs = np.array([1.0, 1.01])
psubs = np.array([400, 410])*10

swelling_distribution = 'field.tavg_viscous_dissipation'
swelling_distribution = 'field.tavg_pos_strain_energy_rate'
param_base = DEFAULT_CASE_PARAM.substitute(
    {
        'vcov': 1.0, 'psub': 400*10, 'dt': 5e-5, 'tf': 0.5,
        'SwellingDistribution': swelling_distribution,
    }
)

case_params_vcov = [param_base.substitute({'vcov': vcov}) for vcov in vcovs]
case_params_psub = [param_base.substitute({'psub': psub}) for psub in psubs]

model = load_model(case_params_vcov[0])

In [ ]:
# NOTE: The swelling factor, v, is defined so that if v=1.1, the total volume of
# the swelling part increases by 10%
dvcov = vcovs[1] - vcovs[0]
dpsub = psubs[1] - psubs[0]

# dSPL/dv
spls = np.array(load_datasets(DATA, 'spl', case_params_vcov))
dspl = spls[1]-spls[0]
dspl_dv = dspl/dvcov

# dSPL/dpsub
spls = np.array(load_datasets(DATA, 'spl', case_params_psub))
dspl = spls[1]-spls[0]
dspl_dpsub = dspl/dpsub

# dphonotrauma/dv
dmgs = np.array(load_datasets(DATA, swelling_distribution, case_params_vcov))
ddmg = dmgs[1]-dmgs[0]
ddmg_dv = ddmg/dvcov

# dphonotrauma/dpsub
dmgs = np.array(load_datasets(DATA, swelling_distribution, case_params_psub))
ddmg = dmgs[1]-dmgs[0]
ddmg_dpsub = ddmg/dpsub

In [ ]:
# The rate of 'psub' change with 'v' is governed by compensation aiming to produce a constant SPL
# The non-linear equation for the SPL should be
# 'spl(v, psub(v)) = constant'
# so the linearized form should satisfy:
# dspl_dv + dspl_dpsub * dpsub_dv = 0
dpsub_dv = -dspl_dv / dspl_dpsub

# Damage induced swelling growth / Healing induced swelling reduction
# Time unit of hours

# For viscous rate, K = 1e-8, K = 1e-7 lead to no VC / VC
K = 1e-8
# For pos. strain energy, K = 1e-11, K = 1e-10 lead to no VC / VC
# K = 1e-6

H = -np.log(0.5)/5
# Vicious cycle growth rate
damage_rate = (ddmg_dpsub*dpsub_dv + ddmg_dv)
growth_rate = K*(ddmg_dpsub*dpsub_dv + ddmg_dv) - H

K_neutral_stability = H / np.max(ddmg_dpsub*dpsub_dv + ddmg_dv) 
print(f"Neutral maximum stability at K = {K_neutral_stability:.2e}")

print(
    "max/min/mean of growth rate: ",
    (np.max(growth_rate), np.min(growth_rate), np.mean(growth_rate))
)

In [ ]:
with h5py.File(f'{post_dir}/postprocess.h5', mode='a') as f:
    key = f'{param_base.to_str()}/field.damage_rate'
    if key not in f:
        f[f'{param_base.to_str()}/field.damage_rate'] = damage_rate

## Parametric swelling study project

In [ ]:
post_dir = 'out'

# This dictionary stores all the post-processed data
DATA = {}

In [ ]:
## Load the main results
with h5py.File(f'{post_dir}/postprocess.h5', mode='r') as f:
    DATA = h5utils.h5_to_dict(f, DATA)

## Load the independence results
# with h5py.File(f'{post_dir}/independence/postprocess.h5', mode='r') as f:
#     DATA = h5utils.h5_to_dict(f, DATA)

In [ ]:
# Get all post-processed cases names
from pprint import pprint
case_names = [key.split('/')[0] for key in DATA.keys() if 'SwellingModel' in key]
case_names = list(set(case_names))
# pprint(case_names)

### Summary of post-processed quantities

In [ ]:
# Specify case names manually

exp_params = []
exp_params += cases.make_exp_params('main_coarse_3D_setup')
exp_params += cases.make_exp_params('main_coarse_3D')
case_names = list(set(exp_param.to_str() for exp_param in exp_params))

In [ ]:
# Get a text summary of what cases were run and what measures were processed
meas_names = [key.split('/')[1] for key in DATA.keys()]
meas_names = list(set(meas_names))

In [ ]:
meas_names

### Additional post-processing

### Stress field at fixed points

In [ ]:
## Post-process stress fields at fixed points
from femvf import static
from femvf.postprocess.solid import StressVonMisesField, StressHydrostaticField

# Loop through each simulation and solve for the fixed point
for case_name in tqdm(case_names):
    # Load the corresponding parameters and appropriate model
    param = cases.ExpParam(case_name)
    model = load_model(param)

    svm_field = StressVonMisesField(model)
    shydro_field = StressHydrostaticField(model)

    state, controls, prop = setup.setup_state_control_prop(param, model, dv=0.02)
    control = controls[0]
    model.set_prop(prop)
    model.set_ini_state(state)
    model.dt = 1e6

    state_fp = setup.setup_ini_state(param, model, dv=0.1)
    svm = svm_field(state_fp, control, prop)
    shydro = shydro_field(state_fp, control, prop)

    # Save the fixed-point von Mises/hydrostatic stresses
    DATA[f'{case_name}/field_fp_vm'] = svm
    DATA[f'{case_name}/field_fp_hydrostatic'] = shydro

### Averaged and integrated field measures 

In [ ]:
_sig_update = {}

for case_name in tqdm(case_names):
    param = cases.ExpParam(case_name)
    model = load_model(param)

    dx = get_dx(model)
    dx_cover = get_dx(model, 'cover')
    _f = dfn.Function(model.solid.residual.form['coeff.prop.emod'].function_space())
    # dofs_cover = meshutils.

    ## Add region averaged measures of stress fields
    stress_field_keys = {'field_tini_vm', 'field_tavg_vm', 'field_fp_vm'}
    stress_field_keys = {'field.tmax_strain_energy', 'field.tavg_viscous_rate'}
    for stress_field_key in stress_field_keys:
        key = f'{case_name}/{stress_field_key}'
        if key in DATA:
            try:
                _f.vector()[:] = DATA[key]
                cover_vol = dfn.assemble(1*dx_cover)
                _sig_update[f'{case_name}/cover_avg_{stress_field_key}'] = dfn.assemble(_f*dx_cover)/cover_vol
                # _sig_update[f'{case_name}/cover_max_{stress_field_key}'] = np.max(_f.vector()[dofs_cover])
                # _sig_update[f'{case_name}/medial_avg_{stress_field_key}'] = dfn.assemble(_f*dx_medial)/medial_vol
                # _sig_update[f'{case_name}/medial_max_{stress_field_key}'] = np.max(_f.vector()[dofs_medial])
            except:
                pass

    ## Add time/space averaged viscous dissipation rate
    t = DATA[f'{case_name}/time.t']
    N = t.size
    wvisc_stats = DATA[f'{case_name}/time.spatial_stats_viscous']
    _sig_update[f'{case_name}/avg_wvisc'] = tavg(wvisc_stats['avg'][-N//2:], t[-N//2:])

    ## Add cover mass and volume
    try:
        fpath = f'{post_dir}/{param.to_str()}.h5'
        prop = setup.setup_basic_prop(param, model)
        if path.isfile(fpath):
            with sf.StateFile(model, fpath, mode='r') as f:
                state_swollen = f.get_state(0)
        else:
            state_swollen, *_ = setup.setup_state_control_prop(param, model, dv=0.02)
        model.set_fin_state(state_swollen)
        model.set_prop(prop)

        # with sf.StateFile(model, f'{POST_DIR}/{case_name}.h5', mode='r') as f:
        #     model.set_fin_state(f.get_state(0))
        #     model.set_prop(f.get_prop())

        dens = model.solid.residual.form['coeff.prop.rho']
        disp = model.solid.residual.form['coeff.state.u1']
        def_grad = ufl.grad(disp)
        j = ufl.det(def_grad + dfn.Identity(3))
        _sig_update[f'{case_name}/mass_cover'] = dfn.assemble(dens*dx_cover)
        _sig_update[f'{case_name}/mass_total'] = dfn.assemble(dens*dx)
        _sig_update[f'{case_name}/vol_cover'] = dfn.assemble(j*dx_cover)
        _sig_update[f'{case_name}/vol_total'] = dfn.assemble(j*dx)
    except RuntimeError:
        _sig_update[f'{case_name}/mass_cover'] = np.nan
        _sig_update[f'{case_name}/mass_total'] = np.nan
        _sig_update[f'{case_name}/vol_cover'] = np.nan
        _sig_update[f'{case_name}/vol_total'] = np.nan

DATA.update(_sig_update)

### Acoustic parameters

In [ ]:
## Add SPL and fundamental frequency to results
for case_name in tqdm(case_names):
    ## Add Fo
    t = DATA[f'{case_name}/time.t']
    gw = DATA[f'{case_name}/time.gw']
    N = gw.size//2
    gw = gw[-N:]
    dt = t[1]-t[0]

    fund_freq, fund_phase, dfreq, dphase, info = \
        modal.fundamental_mode_from_peaks(gw, dt, height=np.max(gw)*0.8)
    # print(fund_freq)
    _sig_update[f'{case_name}/fund_freq'] = fund_freq
    _sig_update[f'{case_name}/dfreq'] = dfreq
    _sig_update[f'{case_name}/period_sample'] = 1/(fund_freq *dt)

    fund_freq, fund_phase, dfreq, dphase, info = \
        modal.fundamental_mode_from_rfft(gw, t[1]-t[0])
    # print(case_name, fund_freq)
    # if fund_freq == np.NaN:
    #     print(case_name)
    _sig_update[f'{case_name}/fund_freq_dft'] = fund_freq
    _sig_update[f'{case_name}/dfreq_dft'] = dfreq

    ## Add SPL
    t = DATA[f'{case_name}/time.t']
    _q = DATA[f'{case_name}/time.q']

    # NOTE: This is done because 'time.q' wasn't computed with an average
    # across all fluid slices before!
    param = ExpParam(case_name)
    q = _q/(param['NZ'])

    # Divide `prms` by 10 to get units of [Pa]
    prms = post.calc_prms(t, q)/10
    _sig_update[f'{case_name}/prms'] = prms
    _sig_update[f'{case_name}/spl'] = 20*np.log10(prms/20e-6)

    ## Add vibration amplitude
    # assert gw.ndim == 1
    _sig_update[f'{case_name}/amplitude'] = gw.max()-gw.min()

DATA.update(_sig_update)

### Plot onset conditions with varying swelling

In [ ]:
DYN_MODEL = load_dynamical_fsi_model(
    mesh_path,
    None,
    SolidType=dsolid.SwellingKelvinVoigtWEpitheliumNoShape,
    FluidType=dfluid.BernoulliAreaRatioSep
)
LDYN_MODEL = load_dynamical_fsi_model(
    mesh_path,
    None,
    SolidType=dsolid.SwellingKelvinVoigtWEpitheliumNoShape,
    FluidType=dfluid.LinearizedBernoulliAreaRatioSep
)
HOPF_MODEL = libhopf.HopfModel(DYN_MODEL, LDYN_MODEL)

In [ ]:
%debug
## Set basic model parameters

# This sets the properties based on the 'main.py' script that runs the sims.
params_dict = DEFAULT_CASE_PARAM.data.copy()
params_dict['vcov'] = 1.1
params_dict['mcov'] = 0
case_param = cases.ExpParam(params_dict)
prop = setup.set_basic_props(DYN_MODEL, case_param)
# prop.print_summary()

# Manually change properties you want to check here
mesh_coords = DYN_MODEL.solid.forms['mesh.mesh'].coordinates()
ymax = mesh_coords[:, 1].max()
ygap = 0.03
prop['ycontact'] = ymax + 9/10*ygap
prop['ymid'] = ymax + ygap
prop['area_lb'] = 2*1/10*ygap
# prop['nu'] = 0.4999
# prop['emod_membrane'] = 0e3*10
# prop['emod'][cells_cover] = 2.5e3*10

# prop.print_summary()

In [ ]:
## Solve for the displacement with swelling
MODEL.set_prop(prop)
nload = 25

# MODEL.solid.control.print_summary()

state_fp, solve_info = solve.solve_static_swollen_config(
    MODEL.solid, MODEL.solid.control, MODEL.solid.prop, nload
)
print(solve_info)
state0 = MODEL.solid.state0.copy()
state0[:] = 0

def assem_res():
    res = dfn.assemble(MODEL.solid.forms['form.un.f1uva'])
    MODEL.solid.forms['bc.dirichlet'].apply(res)
    return res

MODEL.solid.set_fin_state(state0)
print(assem_res().norm('l2'))
MODEL.solid.set_fin_state(state_fp)
print(assem_res().norm('l2'))

statefp_0 = DYN_MODEL.state.copy()
statefp_0[:] = 0
statefp_0['u'] = state_fp['u']

In [ ]:
def solve_fp_r(res, control, prop):
    # Generate a swollen deformation as the initial guess for a fixed-point
    res.set_prop(prop)
    sl_control = res.solid.control
    sl_control[:] = 0

    nload = int(20*np.max(prop['v_swelling'])/1.3)
    sl_state_swollen, solve_info = solve.solve_static_swollen_config(
        res.solid, sl_control, res.solid.prop, nload
    )
    if solve_info['status'] == -1:
        raise RuntimeError(
            "Static swollen deformation could not be computed with info: "
            f"{solve_info}"
        )

    # Use the static swollen state as the initial guess for solving the
    # fixed-point
    xfp_0 = res.state.copy()
    xfp_0[:] = 0
    xfp_0['u'] = sl_state_swollen['u']
    xfp_n, solve_info = libhopf.solve_fp(res, control, prop, xfp_0=xfp_0, psub_incr=1e4)
    if solve_info['status'] == -1:
        raise RuntimeError(
            "Fixed point could not be computed with info: "
            f"{solve_info}"
        )

    return xfp_n, solve_info

In [ ]:
# Quick test to see if `solve_fp_r` works
control = DYN_MODEL.control.copy()
control['psub'] = 400*10
control['psup'] = 0
solve_fp_r(DYN_MODEL, control, prop)

#### Plot growth rate vs psub

In [ ]:
## Compute the growth rate vs. psub
psubs = np.arange(10, 501, 100)*10
control = DYN_MODEL.control.copy()
def set_control(control, psub):
    control['psub'] = psub
    return control
gammas = np.array([
    libhopf.solve_least_stable_mode(
        DYN_MODEL,
        solve_fp_r(DYN_MODEL, set_control(control, psub), prop)[0],
        set_control(control, psub),
        prop
    )[0]
    for psub in tqdm(psubs)
])

In [ ]:
fig, ax = plt.subplots(1, 1)

ax.plot(psubs/10, gammas.real)
ax.axhline(0, ls='-.')
ax_twin = ax.twinx()
ax_twin.plot(psubs/10, gammas.imag/2/np.pi, color=COLOR_CYCLE[1])

ax.set_ylabel("$\omega_\mathrm{real}$", color=COLOR_CYCLE[0])
ax_twin.set_ylabel("$\omega_\mathrm{imag}$ [Hz]", color=COLOR_CYCLE[1])
ax.set_xlabel("$p_\mathrm{sub}$ [Pa]")

#### Onset pressure trend with swelling

In [ ]:
def hopf_from_v(vv, gap_compensation=False):
    params_dict = DEFAULT_CASE_PARAM.data.copy()
    params_dict['vcov'] = vv
    params_dict['mcov'] = 0
    params = ExpParam(params_dict)
    prop = setup.set_basic_props(DYN_MODEL, params)

    # Compute `ymax` when accounting for swelling induced prephonatory gap changes
    DYN_MODEL.set_prop(prop)
    control = DYN_MODEL.solid.control
    control[:] = 0
    state_swollen, info = solve.solve_static_swollen_config(
        DYN_MODEL.solid, control, DYN_MODEL.solid.prop, nload=10
    )
    if gap_compensation:
        ymax = (DYN_MODEL.solid.XREF + state_swollen.sub['u'])[1::2].max()
    else:
        ymax = DYN_MODEL.solid.XREF[1::2].max()

    # Manually change properties here
    ygap = 0.03
    prop['ycontact'] = ymax + ygap - 1/10*ygap
    prop['ymid'] = ymax + ygap
    prop['area_lb'] = 2*1/10*ygap

    psubs = np.arange(10, 301, 100)*10
    xhopf_0 = HOPF_MODEL.state.copy()
    xhopf_0[:] = libhopf.gen_xhopf_0(
        HOPF_MODEL.res, prop, HOPF_MODEL.E_MODE, psubs, solve_fp_r=solve_fp_r
    )

    xhopf, info = libhopf.solve_hopf_by_newton(HOPF_MODEL, xhopf_0, prop)
    return xhopf

def ponset_from_v(vv, gap_compensation=False):
    xhopf = hopf_from_v(vv, gap_compensation)
    return xhopf.sub['psub'][0]

In [ ]:
# %debug
vvs = np.array([1.0, 1.1, 1.2, 1.3])

# vvs = np.array([1.3])
xhopfs_ngap = [hopf_from_v(vv, False) for vv in tqdm(vvs)]
xhopfs_ygap = [hopf_from_v(vv, True) for vv in tqdm(vvs)]

In [ ]:
ponsets_ngap = np.array([xhopf.sub['psub'] for xhopf in xhopfs_ngap])
ponsets_ygap = np.array([xhopf.sub['psub'] for xhopf in xhopfs_ygap])

fonsets_ngap = np.array([xhopf.sub['omega'] for xhopf in xhopfs_ngap])/(2*np.pi)
fonsets_ygap = np.array([xhopf.sub['omega'] for xhopf in xhopfs_ygap])/(2*np.pi)

In [ ]:
DEFAULT_CASE_PARAM

In [ ]:
if FIG_TYPE == 'presentation':
    figsize = (FIG_LX_WIDE, FIG_LY)
else:
    figsize = (FIG_LX, 0.25*FIG_LY)
fig, axs = plt.subplots(1, 2, figsize=figsize)

ax_p = axs[0]
ax_p.plot(vvs, ponsets_ngap/10, label="$\\bar{{m}}'=-0.8$")

ax_f = axs[1]
ax_f.plot(vvs, fonsets_ngap, label="$\\bar{{m}}'=-0.8$")

ax_p.set_xlabel("$v$")
ax_f.set_xlabel("$v$")
ax_p.set_ylabel("$p_\mathrm{on}$ [Pa]")
ax_f.set_ylabel("$f_\mathrm{on}$ [Hz]")
ax_p.legend()
# for ax in axs.flat:
#     ax.legend()

# ax_p.xaxis.set_tick_params(labelbottom=False)

fig.tight_layout()

fig.savefig(f'{FIG_DIR}/OnsetPressureVsSwellingPresentation.{FIG_EXT}')

In [ ]:
if FIG_TYPE == 'presentation':
    figsize = (FIG_LX_WIDE, FIG_LY)
else:
    figsize = (FIG_LX, 0.25*FIG_LY)
fig, axs = plt.subplots(1, 2, figsize=figsize)

ax_p = axs[0]
ax_p.plot(vvs, ponsets_ngap/10, label="variable prephonatory gap")
ax_p.plot(vvs, ponsets_ygap/10, label="constant prephonatory gap")

ax_f = axs[1]
ax_f.plot(vvs, fonsets_ngap, label="variable prephonatory gap")
ax_f.plot(vvs, fonsets_ygap, label="constant prephonatory gap")

ax_p.set_xlabel("$v$")
ax_f.set_xlabel("$v$")
ax_p.set_ylabel("$p_\mathrm{on}$ [Pa]")
ax_f.set_ylabel("$f_\mathrm{on}$ [Hz]")
ax_p.legend()
# for ax in axs.flat:
#     ax.legend()

# ax_p.xaxis.set_tick_params(labelbottom=False)

fig.tight_layout()

fig.savefig(f'{FIG_DIR}/OnsetPressureVsSwelling.{FIG_EXT}')

In [ ]:
dx_cover = get_dx(MODEL, 'cover')
def_grad = ufl.grad(MODEL.solid.forms['coeff.state.u1'])
j = ufl.det(def_grad + dfn.Identity(2))

MODEL.solid.set_fin_state(state0)
v0 = dfn.assemble(j*dx_cover)

MODEL.solid.set_fin_state(state_fp)
v1 = dfn.assemble(j*dx_cover)

print(v0, v1, v1/v0)

#### DEBUG - weird behaviour at high psub for finding fixed points

The fixed point solution seems to be unstable at high $p_{sub}$ for unclear reasons.

In [ ]:
psub = 100*10
control = DYN_MODEL.control.copy()
control['psub'] = psub
xfp, info = libhopf.solve_fp(DYN_MODEL, control, prop, xfp_0=statefp_0, psub_incr=1e4, method='newton')
print(info)
xfp.print_summary()

In [ ]:
DYN_MODEL.set_state(xfp)
DYN_MODEL.set_prop(prop)
DYN_MODEL.fluid.control.print_summary()

xref = DYN_MODEL.solid.forms['mesh.mesh'].coordinates()
u = np.array(xfp.sub['u'])[VERT_TO_VDOF].reshape(-1, 2)
xcur = xref+u

print(f"xmax/ymax = ", xcur[:, 0].max(), xcur[:, 1].max())

In [ ]:
fig, ax = plt.subplots(1, 1)

ax.triplot(xcur[:, 0], xcur[:, 1], MESH.cells())

#### Transient solution near onset

You should see that the transient solution begins self-oscillating around the
pressure where the growth rate > 0  in the above plots.

In [ ]:
# TODO: Add a small snippet to check that the transient model agrees with
# the onset prediction
print(model)
control = model.control.copy()
control[:] = 0
control['psub'] = 200*10
# control.print_summary()
# prop.print_summary()
model.dt = 1e-5
model.set_control(control)
model.set_prop(prop)
state_static, info = static.static_coupled_configuration_picard(model, control, prop)

times_vec = np.arange(0, 0.02, 5e-5)
times = blockvec.BlockVector((times_vec,))
print(times.bshape)

In [ ]:
from femvf.postprocess.base import TimeSeries
from femvf.postprocess.solid import MinGlottalWidth

with sf.StateFile(model, 'temp.h5', mode='w') as f:
    forward.integrate(
        model, f, state_static, [control], prop, times, use_tqdm=True
    )

proc_gw = TimeSeries(MinGlottalWidth(model))
with sf.StateFile(model, 'temp.h5', mode='r') as f:
    gw = proc_gw(f)

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.plot(times_vec, gw)

### Plot glottal width waveform vs. swelling

In [ ]:
vcovs = np.array([1.0, 1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07])
mcov = -0.8

swelling_distribution = 'field.tavg_viscous_rate'
# swelling_distribution = 'field.tmax_strain_energy'

pdefault = DEFAULT_CASE_PARAM.substitute(
    {
        'vcov': 1.0, 'psub': 400*10, 'dt': 5e-5, 'tf': 0.5,
        'SwellingDistribution': swelling_distribution,
    }
)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(0.75*FIG_LX_WIDE, FIG_LY))

case_params = [
    pdefault.substitute({'vcov': vcov, 'mcov': mcov})
    for vcov, mcov in itertools.product(vcovs, [mcov])
]
ts = load_datasets(DATA, 'time.t', case_params)
gws = load_datasets(DATA, 'time.gw', case_params)
fs = load_datasets(DATA, 'fund_freq', case_params)

for vcov, t, gw, f in zip(vcovs, ts, gws, fs):
    idx = slice(0, None)
    t, gw = t[idx], gw[idx]
    fund_freq = f

    ax.plot(
        t, gw, label=(
            f'$v={vcov}, $'
            f'$f_\mathrm{{o}}={fund_freq:.1f} \pm {dfreq:.1e} Hz$'
        )
    )

ax.legend(loc='lower left', bbox_to_anchor=(0, 1))

# ax.set_xlim(0.4, 0.5)
ax.set_xlabel("Time [s]")
ax.set_ylabel("Glottal width [cm]")
fig.tight_layout()

fig.savefig(
    f'{FIG_DIR}/GlottalWidthVsV--{swelling_distribution}.{FIG_EXT}',
    # dpi=200
)

### Animate kinematics for a particular case

In [ ]:
mcov = 0.0
vcovs = [1.0, 1.3]

fnames = [form_fname(MESH_NAME, ECOV, EBOD, vcov, mcov, PSUB) for vcov in vcovs]
print(model.solid.forms['mesh.mesh'].coordinates().shape)

from scipy.signal import find_peaks
from matplotlib import animation

In [ ]:
def tri_to_linexy(tri):
    x, y, edges = (tri.x, tri.y, tri.edges)
    tri_lines_x = np.insert(x[edges], 2, np.nan, axis=1).ravel()
    tri_lines_y = np.insert(y[edges], 2, np.nan, axis=1).ravel()
    return tri_lines_x, tri_lines_y

def plot_update(f, t, offset, artist, dt):
    ii = int(round(t/dt))
    xcur = XREF + f.get_state(offset+ii).sub['u']
    xcur_vert = xcur[VERT_TO_VDOF].reshape(-1, 2)

    # mutate the triplot lines in artists[0]
    tri_frame = tri.Triangulation(xcur_vert[:, 0], xcur_vert[:, 1], CELLS)
    x, y = tri_to_linexy(tri_frame)
    artist[0].set_xdata(x)
    artist[0].set_ydata(y)
    return artist

In [ ]:
XREF_VERT = XREF[VERT_TO_VDOF].reshape(-1, 2)
trian = tri.Triangulation(XREF_VERT[:, 0], XREF_VERT[:, 1], CELLS)

In [ ]:
fig = plt.figure(figsize=(FIG_LX_WIDE, FIG_LY))

gs = mpl.gridspec.GridSpec(2, len(vcovs))
axs_vid = np.array([fig.add_subplot(gs[0, ii]) for ii in range(gs.ncols)])
axs_gw = np.array([fig.add_subplot(gs[1, ii]) for ii in range(gs.ncols)])
axs = np.stack([axs_vid, axs_gw], axis=0)

for ax in axs_vid.flat[1:]:
    axs_vid.flat[0].sharex(ax)
    axs_vid.flat[0].sharey(ax)

for ax in axs_gw[1:]:
    axs_gw.flat[0].sharex(ax)
    axs_gw.flat[0].sharey(ax)

gws = [DATA[f'{fname}/gw'][:] for fname in fnames]
amps = [np.max(gw[-5000:]) for gw in gws]
last_cycle_peaks = [
    find_peaks(gw, height=0.9*amp)[0][-4:]
    for gw, amp in zip(gws, amps)
]

for ax in axs[:, 1:].flat:
    ax.tick_params(labelleft=False)
for ax in axs_vid.flat[1:]:
    ax.tick_params(labelleft=False)

for ax, gw, peaks in zip(axs_gw, gws, last_cycle_peaks):
    ax.plot(gw[peaks[0]:peaks[-1]])

for ax in axs_gw:
    ax.set_xlabel("$t/T$")
axs_gw[0].set_ylabel("Glottal width [cm]")
axs_vid[0].set_ylabel("y [cm]")

artists = [ax.triplot(trian, lw=0.75) for ax in axs_vid]
for ax in axs_vid:
    ax.set_xlabel("x [cm]")
    ax.set_aspect(1)
    ax.set_xlim(-0.1, 0.9)
    ax.set_ylim(0.0, 0.7)

fig.tight_layout()


dts = [3/(peaks[-1]-peaks[0]) for peaks in last_cycle_peaks]
offsets = [peaks[0] for peaks in last_cycle_peaks]



NFRAME = 10
with (
        sf.StateFile(model, f'{OUT_DIR}/{fnames[0]}.h5', mode='r') as f1,
        sf.StateFile(model, f'{OUT_DIR}/{fnames[1]}.h5', mode='r') as f2
    ):
    fs = [f1, f2]

    ii = 1
    plot_update(fs[ii], 0.0, offsets[ii], artists[ii], dts[ii])

    def _update(frame):
        _artists = []
        t = 3*frame/NFRAME
        for f, artist, offset, dt in zip(fs, artists, offsets, dts):
            _artist = plot_update(f, t, offset, artist, dt)
            _artists.append(_artist)

        return np.concatenate(_artists).tolist()

    _update(4)

    ani = animation.FuncAnimation(
        fig,
        _update,
        blit=True,
        frames=np.arange(NFRAME)
    )
    ani.save(f'{FIG_DIR}/v{vcov:.2e}_m{mcov:.2e}.mp4', dpi=300, fps=60)

# for f in fs:
#     f.close()

In [ ]:
## Animate motion
fig, ax = plt.subplots(1, 1, figsize=SIZE_SMALL)
ax.set_xlim(-0.25, 1.0)
ax.set_ylim(0, 0.6)
ax.set_xlabel("x [cm]")
ax.set_ylabel("y [cm]")
ax.set_aspect(1.0, adjustable='box')
ax.axhline(0.55, color='k', ls='-.')
fig.tight_layout()

XREF_VERT = XREF[VERT_TO_VDOF].reshape(-1, 2)
trian = tri.Triangulation(XREF_VERT[:, 0], XREF_VERT[:, 1], CELLS)
artists = ax.triplot(trian, lw=0.75)

vcov = 1.3
fname = form_fname(MESH_NAME, ECOV, EBOD, vcov, mcov, PSUB)
with sf.StateFile(model, f'{OUT_DIR}/{fname}.h5', mode='r') as f:
    def _plot_update(frame):
#         xcur = XREF + f.get_state(frame)['u']
#         xcur_vert = xcur[VERT_TO_VDOF].reshape(-1, 2)

#         # mutate the triplot lines in artists[0]
#         tri_frame = tri.Triangulation(xcur_vert[:, 0], xcur_vert[:, 1], CELLS)
#         x, y = tri_to_linexy(tri_frame)
#         artists[0].set_xdata(x)
#         artists[0].set_ydata(y)
        dt = 1
        offset = 0
        return plot_update(f, frame, offset, artists, dt)
        # return artists
    # plot_update(1000)

    ani = animation.FuncAnimation(
        fig,
        _plot_update,
        blit=True,
        frames=f.size + np.arange(-1000, 0)
    )
    ani.save(f'{FIG_DIR}/v{vcov:.2e}_m{mcov:.2e}.mp4', dpi=450, fps=60)

### Damage measure distribution

In [ ]:
## Specify data arrays
mcov = 0.0

swelling_dist_labels = ('field.tavg_viscous_rate', 'field.tmax_strain_energy')

default_param = DEFAULT_CASE_PARAM.substitute(
    {
        'psub': 400*10, 'ModifyEffect': '', 'tf': 0.5
    }
)

case_param = [{'SwellingDistribution': label} for label in swelling_dist_labels]

model = load_model(default_param)

In [ ]:
dmgs = [
    load_dataset(DATA, swelling_dist, default_param.substitute({'SwellingDistribution': swelling_dist}))
    for swelling_dist in swelling_dist_labels
]

In [ ]:
dmgs[1]

In [ ]:
cells, cell_types, points = make_vtk_mesh_descr(model.solid.residual.mesh())

In [ ]:
plotter = pv.Plotter(shape=(1, 2), border=True, window_size=(2*600, 600))

# grid = pv.UnstructuredGrid(cells, cell_types, points)
grid = pv.UnstructuredGrid(cells, cell_types, points)
for dmg, label in zip(dmgs, swelling_dist_labels):
    grid.cell_data[label] = dmg
    # print(label)

clipped_grid = grid.clip(normal=(0, 0, 1), origin=(0, 0, 0.75))

for n_subplot, label in enumerate(swelling_dist_labels):
    plotter.subplot(0, n_subplot)
    plotter.add_mesh(clipped_grid, show_edges=True, color=None, scalars=label, copy_mesh=True)
    plotter.add_scalar_bar(title=label, position_x=0.1, position_y=0.1)

# plotter.camera_position = [0, 0.5, 1.5]
# plotter.camera.focal_point = [0, 0, 0]

plotter.link_views()
plotter.view_xy()
# plotter.show_bounds()

plotter.show()

### Dynamics over representative cycle

Plot dynamics over a cycle (eg. contact stress over one cycle, maybe average spatial stress field over one cycle)



In [ ]:
measure_names = [
    'ttime', 'gw', 'period_sample',
    'signal_spatial_stats_con_p', 'signal_spatial_stats_con_a',
    'signal_spatial_stats_viscous', 'signal_savg_viscous_rate',
    'signal_spatial_stats_vm'
]
case_params = [DEFAULT_CASE_PARAM.substitute({'vcov': vcov, 'mcov': mcov}) for vcov in VCOVS]
dat = {
    f'{case_param['vcov']:.2f}_{meas_name}': load_dataset(
        DATA, f'{meas_name}', case_param
    )
    for meas_name in measure_names for case_param in case_params
}

# `contact_stats` contains (max pc, avg pc, total, pc, areac) along the last axis

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=SIZE_MED, sharex=True)

for v in VCOVS[::2]:
    _period = dat[f'{v:.2f}_period_sample']
    # print(_period)
    PERIOD = int(round(_period))
    N_PERIODS = 5

    tt = np.linspace(0, N_PERIODS, PERIOD*N_PERIODS)
    yy = dat[f'{v:.2f}_signal_spatial_stats_con_a']['total'][-N_PERIODS*PERIOD:]/10/1e3
    axs[0].plot(tt, yy, label=f"$v$={v:.2f}")

    yy = dat[f'{v:.2f}_signal_spatial_stats_vm']['avg'][-N_PERIODS*PERIOD:]/10/1e3
    axs[1].plot(tt, yy, label=f"$v$={v:.2f}")

ylabels = [
    "$\overline{p_\mathrm{c}}$ [kPa]",
    "$\overline{\sigma_\mathrm{vm}}$ [kPa]"
]
for ax, ylabel in zip(axs, ylabels):
    ax.set_ylabel(ylabel)
axs[-1].set_xlabel("Time [period]")

axs[0].legend()

fig.tight_layout()
fig.savefig(f'{FIG_DIR}/ContactVMCycleDynamicsVsV.{FIG_EXT}')

### Plot fields

In [ ]:
model = MODEL

#### von Mises stress

In [ ]:
# Specify the case(s) of interest

## TODO: Try to ameliorate numerical stress artifacts (likely due to near-incompressibilty)
# mcov = 0.0
meas_names = [
    'field.tavg_vm',
    'field.tavg_hydrostatic',
    'field.tini_vm',
    'field.tini_hydrostatic',
    # 'field_fp_vm',
    # 'field_fp_hydrostatic'
]
if FIG_TYPE == 'manuscript':
    vcovs = [1.0, 1.15, 1.3]
    mcovs = MCOVS[::2]

    vcovs = [1.0, 1.1, 1.2, 1.3]
    mcovs = [0, -0.8, -1.6]
elif FIG_TYPE == 'presentation':
    vcovs = [1.0, 1.1, 1.15, 1.3]
    mcovs = [-0.8]

params = [DEFAULT_CASE_PARAM.substitute({'vcov': vcov}) for vcov in vcovs]
dat = {
    f'{mcov:.2e}/{meas_name}': load_datasets(
        DATA, f'{meas_name}', params, {'mcov': mcov}
    )
    for meas_name in meas_names
    for mcov in mcovs
}

In [ ]:
field_prefix = 'ini'
field_prefix = 'avg'
# field_prefix = 'fp'

figsize = (FIG_LX_WIDE, 13*CM)
# figsize = (8, 4)
# fig, axs = plt.subplots(1, 3, sharex=True, sharey=True, figsize=figsize)

ncol = len(vcovs)
nrow = len(mcovs)
gs = mpl.gridspec.GridSpec(
    nrow+2, ncol+1,
    width_ratios=[1]*ncol + [0.05],
    height_ratios=[0.05]*2+[1]*nrow
)
fig = plt.figure(figsize=figsize)

XREF_VERT = XREF[VERT_TO_VDOF].reshape(-1, 2)
trian = tri.Triangulation(XREF_VERT[:, 0], XREF_VERT[:, 1], CELLS)

axs = np.array(
    [
        [fig.add_subplot(gs[ii+2, jj]) for jj in range(ncol)]
        for ii in range(nrow)
    ],
    dtype=object
)

for ax in axs.flat:
    ax.sharex(axs.flat[0])
    ax.sharey(axs.flat[0])

ax_cbar_abs = fig.add_subplot(gs[0, 0])
ax_cbar_diff = fig.add_subplot(gs[0, 1])

ys = np.array([
    dat[f'{mcov:.2e}/field_tavg_vm'][ii]/10/1e3
    for mcov in mcovs
    for ii in range(len(vcovs))
]).reshape(len(mcovs), len(vcovs), -1)
ys_diff = ys[:, 1:] - ys[:, 0:1]
ys_diff_percent = (ys[:, 1:] - ys[:, 0:1])/ys[:, 0:1]
artists_diff = plot_tripcolor_seq(axs[:, 1:], trian, ys_diff, zmax=0.3)

ys_abs = ys[:, 0]
artists_abs = plot_tripcolor_seq(axs[:, 0], trian, ys_abs, zmax=None)

# for ax, vcov in zip(_axs[1:], vcovs[1:]):
#     ax.text(0.05, 0.95, f"$v={vcov:.2f}$\n$m'={mcov:.2f}$", transform=ax.transAxes, ha='left', va='top')

# ys_abs = np.array([y-ys[0] for y in ys[1:]])
# artists_diff = plot_tripcolor_seq(_axs[1:], trian, ys_diff)

cbar = fig.colorbar(artists_diff, cax=ax_cbar_diff, orientation='horizontal')
ax_cbar_diff.tick_params(top=True, bottom=False, labeltop=True, labelbottom=False)
ax_cbar_diff.set_xlabel(r"$\Delta \tilde{\sigma}_\mathrm{vm}$ [\si{\kPa}]")

cbar = fig.colorbar(artists_abs, cax=ax_cbar_abs, orientation='horizontal')
ax_cbar_abs.tick_params(top=True, bottom=False, labeltop=True, labelbottom=False)
ax_cbar_abs.set_xlabel(r"$\tilde{\sigma}_\mathrm{vm}$ [\si{\kPa}]")

# Add annotations for changing v + m
ax = fig.add_subplot(gs[1, :-1])
ax.axis('off')
ax.annotate(
    "increasing swelling", ha='right', va='center',
    xytext=(0.4, 1.3), xy=(.6, 1.3), xycoords=ax.transAxes,
    arrowprops={'arrowstyle':'simple', 'fc': 'black'}
)

ax = fig.add_subplot(gs[2:, -1])
ax.axis('off')
ax.annotate(
    "increasing softening", ha='center', va='bottom', rotation=-90,
    xytext=(1.15, 0.6), xy=(1.15, 0.4), xycoords=ax.transAxes,
    arrowprops={'arrowstyle':'simple', 'fc': 'black'},
)

for ax in axs.flat:
    ax.set_aspect(1, adjustable='box')

for ax in axs[:, 1:].flat:
    ax.tick_params(axis='y', labelleft=False)

for ax in axs[:-1, :].flat:
    ax.tick_params(axis='x', labelbottom=False)

for ax, mcov in zip(axs[:, 0], mcovs):
    ax.set_ylabel(r"y [\si{\cm}]")
for ax, vcov in zip(axs[-1, :], vcovs):
    ax.set_xlabel(r"x [\si{\cm}]")


for ax, mcov in zip(axs[:, -1], mcovs):
    text_kwargs = {
        'transform': ax.transAxes,
        'ha': 'left',
        'va': 'center'
    }
    ax.text(1, 0.5, f"$\\bar{{m}}'={mcov:.2f}$", rotation=-90, **text_kwargs)
for ax, vcov in zip(axs[0, :], vcovs):
    text_kwargs = {
        'transform': ax.transAxes,
        'ha': 'center',
        'va': 'bottom'
    }
    ax.text(0.5, 1.0, f"\n$v$={vcov:.2f}", **text_kwargs)

fig.tight_layout()
fig.savefig(f'{FIG_DIR}/StressFieldVsV.{FIG_EXT}')

#### Viscous dissipation

In [ ]:
if FIG_TYPE == 'manuscript':
    vcovs = [1.0]
    mcovs = [0.0]

    # vcovs = [1.0, 1.1, 1.2, 1.3]
    # mcovs = [0.0, -0.8, -1.6]
elif FIG_TYPE == 'presentation':
    vcovs = [1.0, 1.1, 1.2, 1.3]
    mcovs = [-0.8, MCOVS[-1]]

meas_name = 'field.tavg_viscous_rate'
params = [DEFAULT_CASE_PARAM.substitute({'vcov': vcov}) for vcov in vcovs]
dat = {
    f'{mcov:.2e}/{meas_name}': load_datasets(
        DATA, f'{meas_name}', params, {'mcov': mcov}
    )
    for mcov in mcovs
}

In [ ]:
figsize = (FIG_LX_WIDE, FIG_LY)
ncol = len(vcovs)
nrow = len(mcovs)
gs = mpl.gridspec.GridSpec(
    nrow+2, ncol+1,
    width_ratios=[1]*ncol + [0.05],
    height_ratios=[0.05]*2+[1]*nrow
)
fig = plt.figure(figsize=figsize)

XREF_VERT = XREF[VERT_TO_VDOF].reshape(-1, 2)
trian = tri.Triangulation(XREF_VERT[:, 0], XREF_VERT[:, 1], CELLS)

axs = np.array(
    [
        [fig.add_subplot(gs[ii+2, jj]) for jj in range(ncol)]
        for ii in range(nrow)
    ],
    dtype=object
)
ax_cbar_abs = fig.add_subplot(gs[0, 0])
ax_cbar_diff = fig.add_subplot(gs[0, 1])

for ax in axs.flat:
    ax.sharex(axs.flat[0])
    ax.sharey(axs.flat[0])

ys = np.array([
    dat[f'{mcov:.2e}/field.tavg_viscous_rate'][ii]*1e-7/(1e-2 ** 2)
    for mcov in mcovs
    for ii in range(len(vcovs))
]).reshape(len(mcovs), len(vcovs), -1)
ys_diff = ys[:, 1:] - ys[:, 0:1]
ys_diff_percent = (ys[:, 1:] - ys[:, 0:1])/ys[:, 0:1]
artists_diff = plot_tripcolor_seq(axs[:, 1:], trian, ys_diff, zmax=50)

ys_abs = ys[:, 0]
artists_abs = plot_tripcolor_seq(axs[:, 0], trian, ys_abs)

cbar = fig.colorbar(artists_diff, cax=ax_cbar_diff, orientation='horizontal')
ax_cbar_diff.tick_params(top=True, bottom=False, labeltop=True, labelbottom=False)
ax_cbar_diff.set_xlabel(r"$\Delta \tilde{w}_\mathrm{visc}$ [\si{\W\m^{-2}}]")

cbar = fig.colorbar(artists_abs, cax=ax_cbar_abs, orientation='horizontal')
ax_cbar_abs.tick_params(top=True, bottom=False, labeltop=True, labelbottom=False)
ax_cbar_abs.set_xlabel(r"$\tilde{w}_\mathrm{visc}$ [\si{\W\m^{-2}}]")

# Add annotations for changing v + m
ax = fig.add_subplot(gs[1, :-1])
ax.axis('off')
ax.annotate(
    "increasing swelling", ha='right', va='center',
    xytext=(0.4, 1.3), xy=(.6, 1.3), xycoords=ax.transAxes,
    arrowprops={'arrowstyle':'simple', 'fc': 'black'}
)

ax = fig.add_subplot(gs[2:, -1])
ax.axis('off')
ax.annotate(
    "increasing softening", ha='center', va='bottom', rotation=-90,
    xytext=(1.15, 0.6), xy=(1.15, 0.4), xycoords=ax.transAxes,
    arrowprops={'arrowstyle':'simple', 'fc': 'black'},
)

for ax in axs.flat:
    ax.set_aspect(1, adjustable='box')

for ax in axs[:, 1:].flat:
    ax.tick_params(axis='y', labelleft=False)

for ax in axs[:-1, :].flat:
    ax.tick_params(axis='x', labelbottom=False)

for ax, mcov in zip(axs[:, 0], mcovs):
    ax.set_ylabel(r"$y$ $[\si{\cm}]$")
for ax, vcov in zip(axs[-1, :], vcovs):
    ax.set_xlabel(r"$x$ $[\si{\cm}]$")

for ax, mcov in zip(axs[:, -1], mcovs):
    text_kwargs = {
        'transform': ax.transAxes,
        'ha': 'left',
        'va': 'center'
    }
    ax.text(1, 0.5, f"$\\bar{{m}}'={mcov:.2f}$", rotation=-90, **text_kwargs)
for ax, vcov in zip(axs[0, :], vcovs):
    text_kwargs = {
        'transform': ax.transAxes,
        'ha': 'center',
        'va': 'bottom'
    }
    ax.text(0.5, 1.0, f"\n$v$={vcov:.2f}", **text_kwargs)

fig.tight_layout()
fig.savefig(f'{FIG_DIR}/ViscousRateVsV.{FIG_EXT}')

In [ ]:
import paraview

#### Viscous dissipation distribution

In [ ]:
if FIG_TYPE == 'manuscript':
    vcovs = [1.0, 1.15, 1.3]
    mcovs = MCOVS[::2]

    vcovs = [1.0, 1.1, 1.2, 1.3]
    mcovs = [0.0, -0.8, -1.6]
elif FIG_TYPE == 'presentation':
    vcovs = [1.3]
    mcovs = [-0.8]

meas_name = 'field_tavg_viscous_rate'
params = [
    DEFAULT_CASE_PARAM.substitute({'vcov': vcov, 'mcov': mcov})
    for vcov, mcov in itertools.product(vcovs, mcovs)
]

dat = load_datasets(
    DATA, f'{meas_name}', params
)

In [ ]:
XREF_VERT = XREF[VERT_TO_VDOF].reshape(-1, 2)
trian = tri.Triangulation(XREF_VERT[:, 0], XREF_VERT[:, 1], CELLS)

gs = mpl.gridspec.GridSpec(2, 1, height_ratios=[0.02, 1])
fig = plt.figure(figsize=(14.7*CM, 13*CM))

ax = fig.add_subplot(gs[1, 0])
artist = ax.tripcolor(trian, dat*1e-7/(1e-2 ** 2))
ax.set_aspect(1)

ax.set_xlabel("$x \, [\mathrm{cm}]$")
ax.set_ylabel("$y \, [\mathrm{cm}]$")

ax_cbar = fig.add_subplot(gs[0, 0])
fig.colorbar(artist, cax=ax_cbar, orientation='horizontal')
ax_cbar.set_xlabel(r"$\tilde{w}_\mathrm{visc}$ $[\mathrm{W}\mathrm{m}^{-2}]$")

fig.savefig(f"{FIG_DIR}/ViscousDissipationExample.{FIG_EXT}")

#### Contact pressure

In [ ]:
## Compute the 's' coordinate for the DG0 contact pressure
forms = MODEL.solid.forms
mesh = forms['mesh.mesh']
facet_function = forms['mesh.facet_function']
facet_data = forms['mesh.facet_label_to_id']
dofmap = forms['fspace.scalar_dg0'].dofmap()

# Find facets along the traction surface and associated DG0 dofs
facet_coords = []
facet_cell_neighbor_idxs = []
for facet in dfn.facets(mesh):
    facet_idx = facet.index()
    if facet_function[facet_idx] == facet_data['pressure']:
        facet_coord = facet.midpoint().array()[:2]
        facet_coords.append(facet_coord)

        cell_neighbor_idx = facet.entities(2)
        assert len(cell_neighbor_idx) == 1
        cell_neighbor_idx = cell_neighbor_idx[0]
        facet_cell_neighbor_idxs.append(cell_neighbor_idx)

facet_coords = np.array(facet_coords)
facet_cell_neighbor_idxs = np.array(facet_cell_neighbor_idxs)
facet_cell_neighbor_dofs = np.array(dofmap.entity_dofs(mesh, 2, facet_cell_neighbor_idxs))

# Sort the DG0 dofs in streamwise increasing order
from femvf.meshutils import sort_vertices_by_nearest_neighbours
idx_sort = sort_vertices_by_nearest_neighbours(facet_coords, np.array([0,0]))
facet_coords = facet_coords[idx_sort]
facet_cell_neighbor_dofs = facet_cell_neighbor_dofs[idx_sort]
_facet_coords = np.concatenate([facet_coords, np.array([[0, 0]])], axis=0)
ds = np.linalg.norm(_facet_coords[1:]-_facet_coords[:-1], axis=-1)
facet_s = np.cumsum(ds)

print(facet_s.shape)
print(facet_cell_neighbor_dofs.shape)

In [ ]:
# mcov = 0.0
mcovs = [0, -0.8, -1.6]
vcovs = [1.0, 1.05, 1.1]

mcovs = [0, -0.8, -1.6]
vcovs = [1.0, 1.1, 1.2, 1.3]

# Load the contact force/area
params = [DEFAULT_CASE_PARAM.substitute({'vcov': vcov}) for vcov in vcovs]
_dat_pcfield = {
    mcov: load_datasets(
        DATA, 'field_tavg_pc', params, {'mcov': mcov}
    )
    for mcov in mcovs
}
_dat_pc = {
    mcov: load_datasets(
        DATA, 'signal_spatial_stats_con_p', params, {'mcov': mcov}
    )
    for mcov in mcovs
}
_dat_pa = {
    mcov: load_datasets(
        DATA, 'signal_spatial_stats_con_a', params, {'mcov': mcov}
    )
    for mcov in mcovs
}
dat_cq = {
    key: np.sum(y['total'] > 0, axis=-1)/y.shape[-1]
    for key, y in _dat_pa.items()
}

# print(_dat_pc[-0.8].shape)
# print(_dat_pc[-0.8].dtype)
# print(_dat_pc[-0.8]['total'].shape)

# Load the contact pressure field and account for averaging over contact times only
# Convert the unit to kpa
# If multiplying by `1/cq`, the contact pressure is averaged over contact only
dat = {
    mcov: _dat_pcfield[mcov]/10/1e3 * 1/dat_cq[mcov][:, None]
    for mcov in mcovs
}

In [ ]:
## PLot contact surface pressure profiles
if FIG_TYPE == 'manuscript':
    fig, axs = plt.subplots(1, 3, figsize=(FIG_LX_WIDE, 8*CM), sharex=True, sharey=True)
elif FIG_TYPE == 'presentation':
    fig, axs = plt.subplots(1, 3, figsize=(FIG_LX_WIDE, FIG_LY), sharex=True, sharey=True)

s = model.fluid.s
for ax, mcov in zip(axs, mcovs):
    print(mcov)
    for vcov, pc, cq, marker in zip(vcovs, dat[mcov], dat_cq[mcov], MARKER_CYCLE):
        kwargs_line = {'marker': marker, 'markevery': 5, 'ms': 5}
        pc_medial = pc[facet_cell_neighbor_dofs]
        ax.plot(facet_s, pc_medial, label=f"$v$={vcov:.2f}", **kwargs_line)

axs[0].set_ylabel(r"$\tilde{p}_\mathrm{c}$ [\si{\kPa}]")
axs[0].legend(loc=(0, 1.05), ncol=1)

# Add annotations for changing v + m
gs = mpl.gridspec.GridSpec(1, 3, figure=fig)
ax = fig.add_subplot(gs[0, :])
ax.axis('off')
ax.annotate(
    "increasing softening", ha='right', va='center',
    xytext=(0.4, 1.3), xy=(.6, 1.3), xycoords=ax.transAxes,
    arrowprops={'arrowstyle':'simple', 'fc': 'black'}
)

for ax, m in zip(axs, mcovs):
    ax.set_xlim(0.6, 1.0)
    ax.set_xlabel("$s$ [\si{\cm}]")
    ax.text(0.05, 0.95, f"$\\bar{{m}}'={m:.2f}$", transform=ax.transAxes, ha='left', va='top')

fig.tight_layout()
fig.savefig(f'{FIG_DIR}/ContactPressureFieldVsV.{FIG_EXT}')

In [ ]:
## PLot contact surface pressure profiles
if FIG_TYPE == 'presentation':
    fig = plt.figure(figsize=(FIG_LX_WIDE, FIG_LY))
elif FIG_TYPE == 'manuscript':
    fig = plt.figure(figsize=(FIG_LX_WIDE, 0.3*FIG_LY))
gs = mpl.gridspec.GridSpec(2, 3, height_ratios=[0.1, 1], hspace=0.05)

ax_legend = fig.add_subplot(gs[0, :])
axs = np.array([fig.add_subplot(gs[1, ii]) for ii in range(gs.ncols)])

for ax in axs:
    ax.sharex(axs[0])
    ax.sharey(axs[0])

ax_legend.set_axis_off()
for ax in axs[1:]:
    ax.tick_params(labelleft=False)

# fig, axs = plt.subplots(1, 3, figsize=(FIG_LX_WIDE, 6*CM), sharex=True, sharey=True)

s = model.fluid.s
xy_med = facet_coords
nn_med = normal(xy_med)
for ax in axs:
    ax.plot(xy_med[:, 0], xy_med[:, 1], color='k')
    ax.plot(xy_med[[0, -1], 0], [0, 0], color='k')
    ax.set_aspect(1)

for ax, mcov in zip(axs, mcovs):
    for vcov, pc in zip(vcovs, dat[mcov]):
        pc_medial = pc[facet_cell_neighbor_dofs]
        scale = -0.2

        pc_n = offset_normal(xy_med, scale*pc_medial[:, None]*nn_med)
        ax.plot(pc_n[:, 0], pc_n[:, 1], label=f"$v$={vcov:.2f}")

        # average contact pressure field
        # ax.plot(s, pc_medial, label=f"$v$={vcov:.1f}")

N = 25
for ax, mcov in zip(axs, mcovs):
    dxy = np.max(dat[mcov])*scale*nn_med[N]
    ax.arrow(*xy_med[N], *dxy, head_width=0)
    pad = np.array([-0.01, 0])
    ax.text(*(xy_med[N] + dxy/4 + pad), f"{np.max(dat[mcov]):.2f} [kPa]", ha='right')
    ax.text(0.05, 0.95, f"$m'={mcov:.2f}$", transform=ax.transAxes, va='top', ha='left')

axs[0].set_ylabel("$y$ [cm]")


labels = [f"$v={v:.2f}$" for v in vcovs]
lines = [mpl.lines.Line2D([0], [0], color=color) for color in COLOR_CYCLE]
ax_legend.legend(lines, labels, loc='lower left', ncol=len(lines))
# axs[0].legend(loc=(0, 1.05), ncol=1, ax=ax_legend)
# axs[0].legend()

for ax in axs:
    ax.set_xlim(0.2, 0.8)
    ax.set_ylim(0.2, 0.7)
    ax.set_xlabel("x [cm]")

fig.tight_layout()
fig.savefig(f'{FIG_DIR}/ContactPressureFieldVsV-Fancy.{FIG_EXT}')

### Contact pressure stats vs $v$

In [ ]:
# Process the trend of contact pressure vs v for each mcov case
DEFAULT_CASE_PARAM = DEFAULT_CASE_PARAM.copy()
mcovs = np.array([0, -0.4, -0.8, -1.2, -1.6])
mcovs = np.array([0])
vcovs = VCOVS[:]

params = [DEFAULT_CASE_PARAM.substitute({'vcov': vcov}) for vcov in vcovs]
stats_contact_p = {
    mcov: load_datasets(
        DATA, 'time.spatial_stats_con_p', params, {'mcov': mcov}
    )
    for mcov in mcovs
}
stats_contact_a = {
    mcov: load_datasets(
        DATA, 'time.spatial_stats_con_a', params, {'mcov': mcov}
    )
    for mcov in mcovs
}
ts = {
    mcov: load_datasets(
        DATA, 'time.t', params, {'mcov': mcov}
    )
    for mcov in mcovs
}
# fund_freqs = {
#     mcov: load_datasets(DATA, 'fund_freq', {'vcov': vcovs, 'mcov': mcov})
#     for mcov in mcovs
# }

In [ ]:
def avg_contact_pressure(t, cp_stats, ca_stats):

    # Assume the final axis varies time
    N = cp_stats.shape[-1]//2

    # Load total/max of contact force + area
    fc = cp_stats['total'][..., -N:]
    ac = ca_stats['total'][..., -N:]
    pc = fc/np.maximum(ac, np.finfo(float).eps)
    t = t[..., -N:]

    # Closed quotient
    cq = np.sum(pc > 0, axis=-1) / N

    # Average contact pressure (during contact)
    tsavg_pc_entire = np.trapz(pc, x=t, axis=-1) / (t[..., -1]-t[..., 0])
    tsavg_pc_contact = tsavg_pc_entire * 1/cq
    return tsavg_pc_contact

# TODO: Make this more general
def plot_contact_vcov_trends(
        axs, vcovs, stats_contact_p, stats_contact_a, ts, fund_freqs,
        add_secondary_axis=False, **plot_kwargs
    ):
    """
    Plot a summary of contact stats given arrays

    Plots a set of contact related statistics as function of swelling.

    Parameters
    ----------
    axs :
        A numpy object array of axes. One statistic is plotted in each axis
    vcovs : ndarray, (m,)
        An array with `m` swelling points
    stats_contact_p, stats_contact_a : ndarray, (m, n)
        An array with `m` swelling points and `n` time points
    """
    N = stats_contact_p.shape[1]//2
    # Load total/max of contact force + area
    smax_pc = stats_contact_p['max'][:, -N:]
    fc = stats_contact_p['total'][:, -N:]

    smax_pa = stats_contact_a['max'][:, -N:]
    ac = stats_contact_a['total'][:, -N:]

    _ac = ac.copy()
    _ac[fc <= 0] = np.inf
    savg_pc = fc/_ac

    t = ts[:, -N:]
    fund_freq = fund_freqs
    period = 1/fund_freq
    # print(savg_pc.shape)

    # Closed quotient
    cq = np.sum(savg_pc > 0, axis=-1) / N
    # axs[2].plot(vcovs, cq, **plot_kwargs)

    # Contact impulse
    # print(ttotal.shape)
    ttotal = t[:, -1] - t[:,0]
    # jc = np.trapz(fc, x=t, axis=-1)
    # # axs[1].plot(vcovs, jc, marker='.', label=f"$m'={mcov:.1f}$")
    # axs[1].plot(vcovs, jc*(period/ttotal), **plot_kwargs)

    # Average contact pressure (during contact)
    tsavg_pc_entire = np.trapz(savg_pc, x=t, axis=-1) / ttotal
    tsavg_pc_contact = tsavg_pc_entire * 1/cq
    axs[0].plot(vcovs, tsavg_pc_contact/10/1e3, **plot_kwargs)
    axs[0].set_ylabel(r"$\hat{p}_\mathrm{c}$ [\si{\kPa}]")

    tsmax_pc_entire = np.max(smax_pc, axis=-1)
    axs[1].plot(vcovs, tsmax_pc_entire/10/1e3, **plot_kwargs)
    axs[1].set_ylabel(r"$\underset{s, t}{\max} \, p_\mathrm{c}$ [\si{\kPa}]")

    # Contact area
    carea_entire = np.sum(ac, axis=-1) / N
    carea_contact = carea_entire * 1/cq
    axs[2].plot(vcovs, carea_contact, **plot_kwargs)
    axs[2].set_ylabel(r"$\tilde{A}_\mathrm{c}$ [\si{\cm}]")

    # Create secondary axis
    if add_secondary_axis:
        _y0s = [
            (tsavg_pc_contact/10/1e3)[0],
            (tsmax_pc_entire/10/1e3)[0],
            carea_contact[0]
        ]
        for ax, y0 in zip(axs, _y0s):
            ax.secondary_yaxis(
                location='right',
                ylabel='$[\si{\percent}]$',
                functions=(
                    lambda x, y0=y0: per_change_forward(x, y0),
                    lambda x, y0=y0: per_change_inv(x, y0)
                )
            )

In [ ]:
if FIG_TYPE == 'manuscript':
    fig, axs = plt.subplots(3, 1, figsize=SIZE_LARGE, sharex=True)
elif FIG_TYPE == 'presentation':
    fig, axs = plt.subplots(1, 3, figsize=(FIG_LX_WIDE, FIG_LY), sharex=True)

for mcov, color, ls in zip(mcovs, COLOR_CYCLE, LS_CYCLE):
    kwargs_line = {
        'ls': ls,
        'color': color
    }
    t = ts[mcov]
    # f0 = fund_freqs[mcov]
    f0 = 1

    if mcov == -0.8:
        add_secondary_axis = True
    else:
        add_secondary_axis = False
    plot_contact_vcov_trends(
        axs, vcovs, stats_contact_p[mcov], stats_contact_a[mcov], t, f0,
        label=f"$\\bar{{m}}'={mcov:.1f}$",
        add_secondary_axis=add_secondary_axis,
        **kwargs_line
    )

for ax in axs:
    ax.set_xticks([1, 1.15, 1.3])

# for label, ax in zip('abcd', axs.flat):
#     ax.text(0.05, 0.95, f"{label})", transform=ax.transAxes, va='top')
axs[-1].set_xlabel("$v$")

# axs[0].legend(loc='lower left', bbox_to_anchor=(0, 1))

axs[0].legend(loc='lower left', ncol=2, bbox_to_anchor=(-0.075, 1.05, 1.15, 0.3), mode='expand')
fig.tight_layout()
fig.savefig(f"{FIG_DIR}/ContactPressureVsV.{FIG_EXT}", dpi=DPI)

### Cover stress vs $v$

In [ ]:
# Process the trend of contact pressure vs v for each mcov case
stress_field_type = 'field_tavg_vm'
# stress_field_type = 'fp_stress_vm'
# stress_field_type = 'ini_stress_vm'

mcovs = np.array([0, -0.8, -1.6])
mcovs = MCOVS

In [ ]:
if FIG_TYPE == 'manuscript':
    fig, axs = plt.subplots(1, 1, figsize=SIZE_SMALL, sharex=True)
elif FIG_TYPE == 'presentation':
    fig, axs = plt.subplots(1, 1, figsize=(FIG_LX_WIDE/2, FIG_LY), sharex=True)
axs = np.atleast_1d(axs)

# mcovs = np.array([0, -0.4, -0.8, -1.2, -1.6])

stat_types = ['avg', 'max']
for ax, stat_type in zip(axs, stat_types):
    if stat_type == 'max':
        region = 'medial'
    else:
        region = 'cover'
    summary_type = f'{region}_{stat_type}'
    params = [DEFAULT_CASE_PARAM.substitute({'vcov': vcov}) for vcov in vcovs]
    data = {
        mcov: load_datasets(
            DATA, f'{summary_type}_{stress_field_type}', params,
            {'mcov': mcov}
        )
        for mcov in mcovs
    }

    for mcov, color, ls in zip(mcovs, COLOR_CYCLE, LS_CYCLE):
        kwargs_line = {
            'ls': ls,
            'color': color
        }
        ys = data[mcov]
        ax.plot(VCOVS, ys/10/1e3, **kwargs_line, label=f"$\\bar{{m}}'={mcov:.1f}$")

    y0 = data[-0.8][0]/10/1e3
    ax.secondary_yaxis(
        location='right',
        ylabel='$[\si{\percent}]$',
        functions=(
            lambda x, y0=y0: per_change_forward(x, y0),
            lambda x, y0=y0: per_change_inv(x, y0)
        )
    )

# if FIG_TYPE == 'manuscript':
#     legend_lines = (
#         [mpl.lines.Line2D([0], [0], color=color, ls=ls) for mcov, color, ls in zip(mcovs, COLOR_CYCLE, LS_CYCLE)]
#         + [mpl.lines.Line2D([0], [0], ls='-', color='k'), mpl.lines.Line2D([0], [0], ls=':', color='k')]
#     )
#     legend_labels = [f"$m'={mcov:.1f}$" for mcov in mcovs] #+ region_types
# elif FIG_TYPE == 'presentation':
#     legend_lines = (
#         [mpl.lines.Line2D([0], [0], color=color, ls=ls) for mcov, color, ls in zip(mcovs, COLOR_CYCLE, LS_CYCLE)]
#         + [mpl.lines.Line2D([0], [0], ls='-', color='k'), mpl.lines.Line2D([0], [0], ls=':', color='k')]
#     )
#     legend_labels = [f"$m'={mcov:.1f}$" for mcov in mcovs]
# axs[0].legend(legend_lines, legend_labels, loc=(0, 1.05))

# axs[0].legend(loc=(0, 1.05))
axs[0].legend(loc='lower left', ncol=2, bbox_to_anchor=(-0.075, 1.05, 1.15, 0.3), mode='expand')

for ax in axs.flat:
    ax.set_xlabel("$v$")

ylabels = [
    r"$\hat{\sigma}_\mathrm{vm}$ [kPa]",
    r"$\hat{\sigma}_\mathrm{vm}$ [kPa]"
]
for ax, ylabel in zip(axs.flat, ylabels):
    ax.set_ylabel(ylabel)
for ax in axs[1:]:
    axs[0].sharey(ax)
# axs[1].set_ylabel(r"$\mathrm{max}\,{\sigma_\mathrm{vm}}$ [kPa]")

fig.tight_layout()
fig.savefig(f"{FIG_DIR}/AvgVMStressVsV_{stress_field_type}.{FIG_EXT}", dpi=DPI)

### Cover viscous dissipation vs. $v$

In [ ]:
VCOVS = np.array([1.0, 1.05, 1.1, 1.2, 1.25])
MCOVS = np.array([0.0, -0.8])

In [ ]:
DEFAULT_CASE_PARAM = DEFAULT_CASE_PARAM.substitute({'ModifyEffect': ''})

params = [DEFAULT_CASE_PARAM.substitute({'vcov': vcov}) for vcov in vcovs]
dat = {
    mcov: load_datasets(
        DATA, 'signal_spatial_stats_viscous', params, {'mcov': mcov}
    )
    for mcov in MCOVS
}

In [ ]:
if FIG_TYPE == 'manuscript':
    fig, ax = plt.subplots(1, 1, figsize=(SIZE_SMALL[0], SIZE_SMALL[1]*0.75))
elif FIG_TYPE == 'presentation':
    fig, ax = plt.subplots(1, 1, figsize=(1/2*FIG_LX_WIDE, FIG_LY))

for mcov, ls, color in zip(MCOVS[:], LS_CYCLE, COLOR_CYCLE):
    kwargs_line = {
        'ls': ls,
        'color': color
    }

    N = dat[mcov].shape[-1]
    ys = np.mean(dat[mcov]['avg'][..., -N//2:], axis=-1)

    ax.plot(VCOVS, ys*1e-7/(1e-2**2), label=f"$\\bar{{m}}'={mcov:.1f}$", **kwargs_line)

ys = np.mean(dat[-0.8]['avg'][..., -N//2:], axis=-1)
y0 = ys[0]*1e-7/(1e-2**2)
ax.secondary_yaxis(
    location='right',
    ylabel='$[\si{\percent}]$',
    functions=(
        lambda x, y0=y0: per_change_forward(x, y0),
        lambda x, y0=y0: per_change_inv(x, y0)
    )
)

ax.set_xlabel("$v$")
ax.set_ylabel(r"$\hat{w}_\mathrm{visc}$ [\si{\W\m^{-2}}]")

# ax.legend()
ax.legend(loc='lower left', ncol=2, bbox_to_anchor=(-0.075, 1.05, 1.15, 0.3), mode='expand')
fig.tight_layout()
fig.savefig(f"{FIG_DIR}/AvgViscRateVsV.{FIG_EXT}", dpi=DPI)

### Average abs Y-Momentum

In [ ]:
DEFAULT_CASE_PARAM = DEFAULT_CASE_PARAM
DEFAULT_CASE_PARAM = DEFAULT_CASE_PARAM.substitute({'ModifyEffect': ''})

params = [DEFAULT_CASE_PARAM.substitute({'vcov': vcov}) for vcov in vcovs]
dat = {
    mcov: load_datasets(
        DATA, 'signal_spatial_state_ymom', params, {'mcov': mcov}
    )
    for mcov in MCOVS
}

dat_constmass = {
    mcov: load_datasets(
        DATA, 'signal_spatial_state_ymom', params, {'mcov': mcov, 'ModifyEffect': 'const_mass'}
    )
    for mcov in MCOVS
}

In [ ]:
if FIG_TYPE == 'manuscript':
    fig, ax = plt.subplots(1, 1, figsize=SIZE_SMALL)
elif FIG_TYPE == 'presentation':
    fig, ax = plt.subplots(1, 1, figsize=(1/2*FIG_LX_WIDE, FIG_LY))

for mcov, color in zip(MCOVS[:], COLOR_CYCLE):
    N = dat[mcov].shape[-1]
    # Plot mean (over time) of abs avg (spatial) y-momentum
    # (over the last 0.5s)
    ys = np.mean(np.abs(dat[mcov]['avg'][..., -N//2:]), axis=-1)
    ax.plot(VCOVS, ys*1e-7/(1e-2**2), label=f"$m'={mcov:.1f}$", ls='-', color=color)

    ys = np.mean(np.abs(dat_constmass[mcov]['avg'][..., -N//2:]), axis=-1)
    ax.plot(VCOVS, ys*1e-7/(1e-2**2), label=f"$m'={mcov:.1f}$ const. mass", ls=':', color=color)

ax.set_xlabel("$v$")
ax.set_ylabel(r"y-momentum")

ax.legend()
fig.tight_layout()
fig.savefig(f"{FIG_DIR}/YMomentum.{FIG_EXT}", dpi=DPI)

### Prephonatory gap vs $v$

In [ ]:
DEFAULT_CASE_PARAM = DEFAULT_CASE_PARAM.substitute({'ModifyEffect': 'const_pregap'})
DEFAULT_CASE_PARAM = DEFAULT_CASE_PARAM.substitute({'ModifyEffect': 'const_mass'})
DEFAULT_CASE_PARAM = DEFAULT_CASE_PARAM.substitute({'ModifyEffect': ''})

params = [DEFAULT_CASE_PARAM.substitute({'vcov': vcov}) for vcov in VCOVS]
dat = {
    mcov: load_datasets(
        DATA, 'gw', params, {'mcov': mcov}
    )
    for mcov in MCOVS
}

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(FIG_LX, 6*CM))

for mcov in MCOVS:
    # Selecting `[:, 0]` gets the initial glottal width, which is the prephonatory gap
    axs[0].plot(VCOVS, dat[mcov][:, 0], label=f"$m'={mcov:.1f}$")

    # Plot the max-min for amplitude
    axs[1].plot(VCOVS, np.max(dat[mcov], axis=-1)-np.min(dat[mcov], axis=-1))

for ax in axs:
    ax.set_xlabel("$v$")

axs[0].set_ylabel("$y_\mathrm{pre}$ [\si{\cm}]")
axs[1].set_ylabel("Amplitude [\si{\cm}]")
axs[0].legend(loc=(0, 1.05))

fig.tight_layout()

fig.savefig(f'{FIG_DIR}/pregap.{FIG_EXT}')

### Damage measures summary

In [ ]:
DEFAULT_CASE_PARAM = DEFAULT_CASE_PARAM

params = [DEFAULT_CASE_PARAM.substitute({'vcov': vcov}) for vcov in VCOVS]
contact_stats_p = {
    mcov: load_datasets(
        DATA, 'signal_spatial_stats_con_p', params,{'mcov': mcov}
    )
    for mcov in MCOVS
}
contact_stats_a = {
    mcov: load_datasets(
        DATA, 'signal_spatial_stats_con_a', params,{'mcov': mcov}
    )
    for mcov in MCOVS
}
times = {
    mcov: load_datasets(
        DATA, 'ttime', params, {'mcov': mcov}
    )
    for mcov in MCOVS
}
avg_pc_data = {
    mcov: np.array([
        avg_contact_pressure(t, p, a) for
        t, p, a in zip(times[mcov], contact_stats_p[mcov], contact_stats_a[mcov])
    ])
    for mcov in MCOVS
}
avg_vm_data = {
    mcov: load_datasets(
        DATA, 'cover_avg_field_tavg_vm', params, {'mcov': mcov}
    )
    for mcov in MCOVS
}
avg_wvisc_data = {
    mcov: load_datasets(
        DATA, 'avg_wvisc', params,{'mcov': mcov}
    )
    for mcov in MCOVS
}

In [ ]:
fig = plt.figure(figsize=(FIG_LX_WIDE, FIG_LY), constrained_layout=True)
gs = mpl.gridspec.GridSpec(1, 3, figure=fig)

axs = np.array([fig.add_subplot(gs[0, jj]) for jj in range(gs.ncols)])

ydatas = [avg_wvisc_data, avg_pc_data, avg_vm_data]
ylabels = [
    r"$\hat{w}_\mathrm{visc}$ [\si{\W\m^{-2}}]",
    r"$\hat{p}_\mathrm{c}$ [\si{\kPa}]",
    r"$\hat{\sigma}_\mathrm{vm}$ [kPa]"
]
for ax, ylabel in zip(axs, ylabels):
    ax.set_ylabel(ylabel)
    ax.set_xlabel("$v$")

for ax, ydata in zip(axs, ydatas):
    for mcov, ls, color in zip(MCOVS, LS_CYCLE, COLOR_CYCLE):
        ax.plot(VCOVS, ydata[mcov], label=f"$m'=${mcov:.1f}", ls=ls, color=color)

axs[0].legend(loc='lower left', ncol=2, bbox_to_anchor=(-0.075, 1.05, 1.15, 0.3), mode='expand')

n = 4
v = VCOVS[n]
for ax, ydata in zip(axs, ydatas):
    ys = [ydata[mcov][n] for mcov in (0, -1.6)]
    annotate_vertical_trend(ax, "increasing softening", v, *ys)

fig.savefig(f'{FIG_DIR}/DamageMeasureSummary.{FIG_EXT}')

### Compare w/ and w/o specific mechanisms

In [ ]:
PREFIX_GAP = 'const_pregap'
PREFIX_MASS = 'const_mass'
PREFIX_GAPMASS = 'const_mass_pregap'

In [ ]:
DEFAULT_CASE_PARAM = DEFAULT_CASE_PARAM.substitute({'mcov': -0.8})
def load_sub_effects(data, measure_name, DEFAULT_EXP_PARAM=DEFAULT_CASE_PARAM):
    params = [
        DEFAULT_EXP_PARAM.substitute({'vcov': vcov, 'ModifyEffect': ''})
        for vcov in VCOVS
    ]
    ret_data = {}
    ret_data['default'] = load_datasets(
        DATA, measure_name, params
    )

    ret_data['const_stiff'] = load_datasets(
        DATA, measure_name, params, {'mcov': 0.0, 'ModifyEffect': ''}
    )

    ret_data['const_mass'] = load_datasets(
        DATA, measure_name, params, {'ModifyEffect': PREFIX_MASS}
    )

    ret_data['const_pregap'] = load_datasets(
        DATA, measure_name, params, {'ModifyEffect': PREFIX_GAP},
        DEFAULT_EXP_PARAM
    )

    ret_data['const_mass_pregap_stiffness'] = load_datasets(
        DATA, measure_name, params, {'ModifyEffect': PREFIX_GAPMASS, 'mcov': 0}
    )
    return ret_data

effectname_to_ylabel = {
    'default': "all swelling effects",
    'const_stiff': "no stiffness change",
    'const_mass': "no mass change",
    'const_pregap': "no gap change",
    'const_mass_pregap_stiffness': "no stiffness, mass, or gap change"
}

#### Subeffects summary

In [ ]:
times = load_sub_effects(DATA, 'ttime')
contact_stats_p = load_sub_effects(DATA, 'signal_spatial_stats_con_p')
contact_stats_a = load_sub_effects(DATA, 'signal_spatial_stats_con_a')

contact_p = {
    key: [
        avg_contact_pressure(t, cp, ca)
        for t, cp, ca in zip(times[key], contact_stats_p[key], contact_stats_a[key])
    ]
    for key in contact_stats_p.keys()
}

avg_vm_data = load_sub_effects(DATA, 'cover_avg_field_tavg_vm')

avg_wvisc_data = load_sub_effects(DATA, 'avg_wvisc')

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=SIZE_LARGE, sharex=True)

ax = axs[0]
for (effect_name, ys), ls in zip(avg_vm_data.items(), LS_CYCLE):
    ax.plot(VCOVS, np.array(ys)/10/1e3, label=effectname_to_ylabel[effect_name], ls=ls)
ax.set_ylabel(r"$\hat{\sigma}_\mathrm{vm}$ [\si{\kPa}]")

ax = axs[1]
for (effect_name, ys), ls in zip(avg_wvisc_data.items(), LS_CYCLE):
    ax.plot(VCOVS, np.array(ys)*1e-7 / (1e-2)**2, label=effectname_to_ylabel[effect_name], ls=ls)
ax.set_ylabel(r"$\hat{w}_\mathrm{visc}$ $[\si{\W\m^{-2}}]$")

ax = axs[2]
for (effect_name, ys), ls in zip(contact_p.items(), LS_CYCLE):
    ax.plot(VCOVS, np.array(ys)/10/1e3, label=effectname_to_ylabel[effect_name], ls=ls)
ax.set_ylabel(r"$\hat{p}_\mathrm{c}$ [\si{\kPa}]")

_y0s = [
    np.array(avg_vm_data['default'])[0]/10/1e3,
    np.array(avg_wvisc_data['default'])[0]*1e-7 / (1e-2)**2,
    np.array(contact_p['default'])[0]/10/1e3
]
for ax, y0 in zip(axs, _y0s):
    ax.secondary_yaxis(
        location='right',
        ylabel='$[\si{\percent}]$',
        functions=(
            lambda x, y0=y0: per_change_forward(x, y0),
            lambda x, y0=y0: per_change_inv(x, y0)
        )
    )

# for label, ax in zip('abcd', axs.flat):
#     ax.text(0.05, 0.95, f"{label})", transform=ax.transAxes, va='top')

# axs[0].legend(loc='lower left', bbox_to_anchor=(0, 1))
axs[0].legend(loc='lower left', bbox_to_anchor=(-0.075, 1.05, 1.15, 0.3), mode='expand')

axs[-1].set_xlabel("$v$")
fig.tight_layout()

fig.savefig(f"{FIG_DIR}/SwellingSubeffects.{FIG_EXT}")

#### F0

In [ ]:
# Load the F0 info with gap compensation
data_f0 = load_sub_effects(DATA, 'fund_freq')

In [ ]:
## Compare F0 w/ and w/o gap compensation
fig, ax = plt.subplots(1, 1, figsize=SIZE_SMALL, sharey=True)

for mcov in MCOVS[:1]:
    # F0 (w/o gap)
    f0s = data_f0['default']
    ax.plot(VCOVS, f0s, marker='.', label=f"standard")

    # F0 (w/ gap)
    f0s = data_f0['const_pregap']
    ax.plot(VCOVS, f0s, marker='.', label=f"$m'={mcov:.1f}$ const. gap")

    # F0 (w/ gap)
    f0s = data_f0['const_mass']
    ax.plot(VCOVS, f0s, marker='.', label=f"$m'={mcov:.1f}$ const. mass")

ax.set_xlabel("$v$ [1]")
ax.set_ylabel("$F_0$ [Hz]")

ax.legend()
fig.tight_layout()
fig.savefig(f"{FIG_DIR}/F0GapCompensationComparisonVsV.png", dpi=DPI)

#### Contact pressure

In [ ]:
## Load the contact pressure info with gap compensation
prefixes = ['', PREFIX_GAP, PREFIX_MASS, PREFIX_GAPMASS]
mcov = -0.8
DEFAULT_CASE_PARAM = DEFAULT_CASE_PARAM.substitute({'mcov': -0.8})
print(DEFAULT_CASE_PARAM)

params = [DEFAULT_CASE_PARAM.substitute({'vcov': vcov}) for vcov in vcovs]
contact_stats_p = [
    load_datasets(
        DATA, 'signal_spatial_stats_con_p', params,
        {'ModifyEffect': prefix}
    )
    for prefix in prefixes
]

contact_stats_a = [
    load_datasets(
        DATA, 'signal_spatial_stats_con_a', params,
        {'ModifyEffect': prefix}
    )
    for prefix in prefixes
]

ts = [
    load_datasets(
        DATA, 'ttime', params,
        {'ModifyEffect': prefix}
    )
    for prefix in prefixes
]

fund_freqs = [
    load_datasets(
        DATA, 'fund_freq', params,
        {'ModifyEffect': prefix}
    )
    for prefix in prefixes
]

vcovs = VCOVS

In [ ]:
## Compare mean contact pressure w/ and w/o gap compensation
if FIG_TYPE == 'manuscript':
    fig, axs = plt.subplots(3, 1, figsize=(FIG_LX, 0.7*FIG_LY), sharex=True)
elif FIG_TYPE == 'presentation':
    fig, axs = plt.subplots(1, 3, figsize=(FIG_LX_WIDE, FIG_LY), sharex=True)

postfixes = ['normal', 'const. gap', 'const. mass', 'const. gap + mass']
for _cp, _ca, _ts, _fund_freqs, postfix in zip(contact_stats_p, contact_stats_a, ts, fund_freqs, postfixes):
    plot_kwargs = {
        'marker': '.',
        # 'label': f"$m'={mcov:.1f}$"+postfix
        'label': postfix
    }
    plot_contact_vcov_trends(axs, vcovs, _cp, _ca, _ts, _fund_freqs, **plot_kwargs)

if FIG_TYPE == 'manuscript':
    ylabels = [
        "$p_\mathrm{c}$ [kPa]\n(over contact)",
        r"$J_\mathrm{c}$ [$1 \times 10^{-5} \mathrm{N s}$]"+"\n(per cycle)"
    ]
elif FIG_TYPE == 'presentation':
    ylabels = [
        "$\mathrm{mean} \, p_\mathrm{c}$ [kPa]\n(over contact)",
        "$\mathrm{max} \, p_\mathrm{c}$ [kPa]",
        r"$A_\mathrm{c}$ [cm]"
    ]
# for ax, ylabel in zip(axs.flat, ylabels):
#     ax.set_ylabel(ylabel)

# for ax in axs[1:]:
#     axs[0].sharey(ax)

for ax in axs:
    ax.set_xlabel("$v$")

axs[0].legend(loc='lower left', bbox_to_anchor=(0, 1))
fig.tight_layout()
fig.savefig(f"{FIG_DIR}/ContactPressureGapCompensationComparisonVsV.{FIG_EXT}")

#### von Mises stress

In [ ]:
## Load the mean von Mises stress data w/ gap compensation
params = [DEFAULT_CASE_PARAM.substitute({'vcov': vcov}) for vcov in VCOVS]
data_cover_avg_vm = {
    mcov: load_datasets(
        DATA, 'cover_avg_field_tini_vm', params,{'mcov': mcov}
    )
    for mcov in MCOVS[:1]
}
data_cover_avg_vm_gap = {
    mcov: load_datasets(
        DATA, 'cover_avg_field_tini_vm', params,
        {'mcov': mcov, 'ModifyEffect': PREFIX_GAP}
    )
    for mcov in MCOVS[:1]
}
data_cover_avg_vm_mass = {
    mcov: load_datasets(
        DATA, 'cover_avg_field_tini_vm', params,
        {'mcov': mcov, 'ModifyEffect': PREFIX_MASS}
    )
    for mcov in MCOVS[:1]
}

data_cover_avg_vm = {
    mcov: load_datasets(
        DATA, 'cover_avg_field_tavg_vm', params, {'mcov': mcov}
    )
    for mcov in MCOVS[:1]
}
data_cover_avg_vm_gap = {
    mcov: load_datasets(
        DATA, 'cover_avg_field_tavg_vm', params,
        {'mcov': mcov, 'ModifyEffect': PREFIX_GAP}
    )
    for mcov in MCOVS[:1]
}
data_cover_avg_vm_mass = {
    mcov: load_datasets(
        DATA, 'cover_avg_field_tavg_vm', params,
        {'mcov': mcov, 'ModifyEffect': PREFIX_MASS}
    )
    for mcov in MCOVS[:1]
}

In [ ]:
## Compare mean von Mises stress w/ and w/o gap compensation
fig, ax = plt.subplots(1, 1, figsize=SIZE_MED_WIDE, sharey=True)

for mcov in MCOVS[:1]:
    # Avg. cover vm stress (w/o gap)
    vm_stresses = data_cover_avg_vm[mcov]
    ax.plot(VCOVS, vm_stresses/10/1e3, marker='.', label=f"$m'={mcov:.1f}$")

    # Avg. cover vm stress (w/ gap)
    vm_stresses = data_cover_avg_vm_gap[mcov]
    ax.plot(VCOVS, vm_stresses/10/1e3, marker='.', label=f"$m'={mcov:.1f}$ w/ gap")

    # Avg. cover vm stress (w/ mas)
    vm_stresses = data_cover_avg_vm_mass[mcov]
    ax.plot(VCOVS, vm_stresses/10/1e3, marker='.', label=f"$m'={mcov:.1f}$ w/ mass")

ax.set_xlabel("$v$ [1]")

ax.set_ylabel("$\sigma_\mathrm{vm}$ [kPa]")

ax.legend()
fig.tight_layout()
fig.savefig(f"{FIG_DIR}/VMGapCompensationComparisonVsV.{FIG_EXT}", dpi=DPI)

#### Viscous dissipation

#### SPL

In [ ]:
params = [DEFAULT_CASE_PARAM.substitute({'vcov': vcov}) for vcov in VCOVS]

prms_trends = {
    mcov: load_datasets(
        DATA, 'prms', params,
        {'mcov': mcov})
    for mcov in MCOVS[:1]
}
prms_trends_gap = {
    mcov: load_datasets(
        DATA, 'prms', params,
        {'mcov': mcov, 'ModifyEffect': PREFIX_GAP}
    )
    for mcov in MCOVS[:1]
}
prms_trends_mass = {
    mcov: load_datasets(
        DATA, 'prms', params,
        {'mcov': mcov, 'ModifyEffect': PREFIX_MASS}
    )
    for mcov in MCOVS[:1]
}

In [ ]:
## Compare mean SPL
fig, ax = plt.subplots(1, 1, figsize=SIZE_MED_WIDE, sharey=True)

for mcov in MCOVS[:1]:
    # Avg. cover vm stress (w/o gap)
    y = prms_trends[mcov]
    ax.plot(VCOVS, 20*np.log10(y/20e-6), marker='.', label=f"$m'={mcov:.1f}$")

    # Avg. cover vm stress (w/ gap)
    y = prms_trends_gap[mcov]
    ax.plot(VCOVS, 20*np.log10(y/20e-6), marker='.', label=f"$m'={mcov:.1f}$ w/ gap")

    # Avg. cover vm stress (w/ gap)
    y = prms_trends_mass[mcov]
    ax.plot(VCOVS, 20*np.log10(y/20e-6), marker='.', label=f"$m'={mcov:.1f}$ w/ mass")

ax.set_xlabel("$v$ [1]")
ax.set_ylabel("$SPL$ [dB]")

ax.legend()
fig.tight_layout()
fig.savefig(f"{FIG_DIR}/SPLGapCompensationComparisonVsV.png", dpi=DPI)

### 3D Mesh independence

In [ ]:
DEFAULT_CASE_PARAM = DEFAULT_CASE_PARAM.substitute({
    'vcov': 1,
    'mcov': 0,
    'SwellingModel': 'power'
})

clscales = np.array([0.94, 0.75, 0.6, 0.47, 0.38])
nzs = np.array([12, 15, 19, 24, 30])
params = [
    DEFAULT_CASE_PARAM.substitute({'clscale': clscale, 'NZ': nz})
    for clscale, nz in zip(clscales, nzs)
]

wvisc = load_datasets(
    DATA, 'avg_wvisc', params,
)
fo = load_datasets(
    DATA, 'fund_freq', params
)
prms = load_datasets(
    DATA, 'prms', params
)

cell_counts = nzs/15 * (0.75/np.array(clscales))**2

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(FIG_LX, FIG_LY), sharex=True)

axs[0].plot(cell_counts, prms)
axs[0].set_ylabel("$p_\mathrm{rms}$")

axs[1].plot(cell_counts, fo)
axs[1].set_ylabel("$f_\mathrm{o}$")

axs[2].plot(cell_counts, wvisc)
axs[2].set_ylabel("$P_\mathrm{visc}$")
axs[2].set_xlabel("$K_\mathrm{cell}$")
axs[2].set_xticks(cell_counts)
axs[2].xaxis.set_tick_params(rotation=-60)

fig.savefig(f'{FIG_DIR}/3DMeshIndependence.svg')

### Mesh and timestep independence

In [ ]:
## Load mesh independence simulation data

# NOTE: For h independence,`dt` can be either 1.25e-5 or 6.25e-6
DEFAULT_CASE_PARAM = DEFAULT_CASE_PARAM.substitute({
    'MeshName': 'M5_CB_GA3_SplitCover',
    'clscale': 0.5,
    'mcov': -1.6,
    'vcov': 1.3,
    'tf': 0.5
})

signals = {}
# with h5py.File('out_independence/postprocess.h5', mode='r') as f:
with h5py.File('out/postprocess.h5', mode='r') as f:
    h5utils.h5_to_dict(f, signals)

independence_case = 'dt'
independence_case = 'mesh'
independence_case = 'all'
if independence_case == 'mesh':
    dts = [6.25e-6]
    clscales = np.array([2, 1, .5])
elif independence_case == 'dt':
    dts = 5e-5 * 2.0**np.arange(-1, -5, -1)
    clscales = [1]
elif independence_case == 'all':
    dts = 5e-5 * 2.0**np.arange(0, -5, -1)
    clscales = np.array([2, 1, .5])
params_trend = [{'dt': dt, 'clscale': clscale} for dt, clscale in itertools.product(dts, clscales)]
# p_trend = iter_params(params_trend, DEFAULT_EXP_PARAM)

measure_names = [
    'signal_spatial_stats_viscous',
    'signal_spatial_stats_con_p',
    'signal_spatial_stats_con_a',
    'signal_spatial_stats_vm'
]
data_measures = {
    key: load_datasets(
        signals, key,
        params_trend,
        default_exp_param=DEFAULT_CASE_PARAM
    )
    for key in measure_names
}

data_measures['signal_contact_p'] = [
    f['total']/np.maximum(a['total'], np.finfo(float).eps)
    for f, a in zip(
        data_measures['signal_spatial_stats_con_p'],
        data_measures['signal_spatial_stats_con_a']
    )
]

ys = data_measures['signal_spatial_stats_con_a']
cqs = np.array([np.sum(y['total'] > 0)/y['total'].size for y in ys])

In [ ]:
## Plot trends in a selection of measures vs. the refinement factor
# This is done for either mesh or time step refinement (see `independence_case`)

ylabels = {
    'savg_tavg_vm': r"$\hat{\sigma}_\mathrm{vm}$ [\si{\kPa}]",
    'savg_tavg_viscous_rate': r"$\hat{w}_\mathrm{visc}$ [\si{\W\cm^{-2}}]",
    'savg_contact_pressure': r"$\hat{p}_\mathrm{c}$ [\si{\kPa}]",
    'savg_tavg_contact_a': r"$\tilde{A}_\mathrm{c}$ [\si{\cm^2}]",
    'savg_tavg_contact_p': "Avg. contact force [dyne]",
    'savg_tmax_contact_p': "Max. contact force [dyne]",
    'savg_tmax_contact_a': "Max. contact area [cm]",
}
keys = list(ylabels.keys())

data = {}

ys = data_measures['signal_spatial_stats_vm']
data['savg_tavg_vm'] = [np.mean(y['avg'][y.size//2:]) for y in ys]

ys = data_measures['signal_spatial_stats_viscous']
data['savg_tavg_viscous_rate'] = np.array([np.mean(y['avg'][y.size//2:]) for y in ys]) * 1e-7/((1e-2)**2)

ys = data_measures['signal_spatial_stats_con_p']
data['savg_tavg_contact_p'] = [np.mean(y['total'][y.size//2:]) for y in ys]

ys = data_measures['signal_spatial_stats_con_a']
data['savg_tavg_contact_a'] = [np.mean(y['total'][y.size//2:]) for y in ys]

ys = data_measures['signal_spatial_stats_con_p']
data['savg_tmax_contact_p'] = [np.max(y['total'][y.size//2:]) for y in ys]

ys = data_measures['signal_spatial_stats_con_a']
data['savg_tmax_contact_a'] = [np.max(y['total'][y.size//2:]) for y in ys]

ys = data_measures['signal_contact_p']
data['savg_contact_pressure'] = np.array([np.mean(y[y.size//2:]) for y in ys])*1/cqs/10/1e3

In [ ]:
def plot_refinement(
        fig,
        axs,
        xdata,
        ydata,
        zdata,
        ylabel_formatter=None,
        ymarkers=None,
        **plot_kwargs
    ):
    """
    Plot a 3D surface 'z = f(x, y)' as a series of 'z=g(x)' plots for each 'y'

    Parameters
    ----------
    fig : mpl.Figure
    axs : List[mpl.ax.Axes]
    labels : List[str]
    xdata : np.ndarray (n,)
    ydata : np.ndarray (m,)
    zdata : Mapping[str, np.ndarray(n, m)]
    """
    if ymarkers is None:
        ymarkers = ['']*ydata.size

    if ylabel_formatter is None:
        def ylabel_formatter(y):
            return f"{y}"
    for (key, zs), ax in zip(zdata.items(), axs.flat):
        assert zs.shape == (xdata.size, ydata.size)

        for z, y, marker in zip(zs.T, ydata, ymarkers):
            ax.plot(xdata, z, label=ylabel_formatter(y), marker=marker, **plot_kwargs)

#### Time step refinement

In [ ]:
## Plot vs time refinement
if FIG_TYPE == 'presentation':
    fig, axs = plt.subplots(2, 2, sharex=True, figsize=(FIG_LX_WIDE, 3))
else:
    fig, axs = plt.subplots(4, 1, sharex=True, figsize=(FIG_LX, 5))
axs_twin = np.array([ax.twinx() for ax in axs.flat]).reshape(axs.shape)

shape = (len(dts), len(clscales))
zdata = {key: np.reshape(data[key], shape) for key in keys}
zdata_err = {key: 100*np.abs((y-y.flat[-1])/y.flat[-1]) for key, y in zdata.items()}

xs = 5e-5/dts
def ylabel_formatter(clscale):
    return f"Mesh refinement: {1/clscale:.2f}"
plot_refinement(
    fig, axs, 5e-5/dts, clscales, zdata,
    ymarkers=MARKER_CYCLE,
    ylabel_formatter=ylabel_formatter
)
plot_refinement(
    fig, axs_twin, 5e-5/dts, clscales, zdata_err, ls=':',
    ymarkers=MARKER_CYCLE,
    ylabel_formatter=ylabel_formatter
)

for key, ax, ax_twin in zip(keys, axs.flat, axs_twin.flat):
    ax.set_ylabel(ylabels[key])
    ax_twin.set_ylabel("Error $[\si{\percent}]$")

from matplotlib import ticker
for ax, ax_twin in zip(axs.flat, axs_twin.flat):
    ax.set_xscale('log')
    ax.xaxis.set_major_formatter(ticker.ScalarFormatter())
    ax.xaxis.set_minor_formatter(ticker.NullFormatter())
    ax.xaxis.set_tick_params(which='minor', bottom=False)
    ax.set_xticks(xs)
    # ax_twin.set_yscale('log')
    ax_twin.set_ylim(0, 10)

xlabel = "$\Delta t$ refinement factor"
if FIG_TYPE == 'presentation':
    for ax in axs[-1, :]:
        ax.set_xlabel(xlabel)
else:
    axs[-1].set_xlabel(xlabel)

# axs.flat[0].legend(loc=(0, 1))
axs.flat[0].legend(loc='lower left', bbox_to_anchor=(-0.075, 1.05, 1.15, 0.3), mode='expand')
fig.tight_layout()
fig.savefig(f"{FIG_DIR}/Independence-{independence_case}.{FIG_EXT}")

#### Mesh refinement

In [ ]:
## Plot vs mesh refinement
fig, axs = plt.subplots(3, 2, sharex=True, figsize=(FIG_LX_WIDE, 10))
axs_twin = np.array([ax.twinx() for ax in axs.flat]).reshape(axs.shape)

shape = (len(dts), len(clscales))
zdata = {key: np.reshape(data[key], shape).T for key in keys}
zdata_err = {key: 100*np.abs((y-y.flat[-1])/y.flat[-1]) for key, y in zdata.items()}

xs = 1/clscales
plot_refinement(fig, axs, xs, dts, zdata)
plot_refinement(fig, axs_twin, xs, dts, zdata_err, ls=':')

for key, ax, ax_twin in zip(keys, axs.flat, axs_twin.flat):
    ax.set_ylabel(ylabels[key])
    ax_twin.set_ylabel("Error [%]")

from matplotlib import ticker
for ax, ax_twin in zip(axs.flat, axs_twin.flat):
    ax.set_xscale('log')
    ax.xaxis.set_major_formatter(ticker.ScalarFormatter())
    ax.xaxis.set_minor_formatter(ticker.NullFormatter())
    ax.xaxis.set_tick_params(which='minor', bottom=False)
    ax.set_xticks(xs)
    # ax_twin.set_ylim(0, 10)

xlabel = "$h$ Refinement factor"
if FIG_TYPE == 'presentation':
    for ax in axs[-1, :]:
        ax.set_xlabel(xlabel)
else:
    axs.flat[-1].set_xlabel(xlabel)

axs.flat[0].legend()
fig.tight_layout()
fig.savefig(f"{FIG_DIR}/Independence-{independence_case}.{FIG_EXT}")

In [ ]:
times = load_datasets(
    signals, 'ttime',
    params_trend,
    default_exp_param=DEFAULT_CASE_PARAM
)
gaws = load_datasets(
    signals, 'gw',
    params_trend,
    default_exp_param=DEFAULT_CASE_PARAM
)

In [ ]:
fig, ax = plt.subplots(1, 1)
for t, gw in zip(times, gaws):
    ax.plot(t, gw)

ax.set_xlabel("Time [s]")
ax.set_ylabel("GAW [cm]")
ax.legend()

### Yang (2017) m' calculation

The below plot's data from Yang et al. - Quantitative Study of the Effects of Dehydration on the Viscoelastic Parameters in the Vocal Fold Mucosa (2017).
The data are from Tables 2 and 3 which measure 1D moduli (stress/strain) during loading and unloading phases, respectively.


In [ ]:
# Assume that the dehydration % in the study represents the percent
# of original water mass lost
dehydrations = np.array([0, .4, .6, .8])

# Calculate changes in volume for an initial unit volume of VF material
initial_water_vol = 0.8
initial_solid_vol = 0.2
water_vols = initial_water_vol * (1-dehydrations)
total_vols = (water_vols+initial_solid_vol)
hydrations = (water_vols)/total_vols
vs = total_vols/total_vols[0]

In [ ]:
# From table 2 (stretch/loading)
mods_stretch = np.array([41374.5, 44846.05, 50198.98, 78157.41])
# From table 3 (release/unloading)
mods_release = np.array([23589.36, 29810.85, 23198.20, 22833.13])
mods_avg = (mods_stretch+mods_release)/2

fig, axs = plt.subplots(3, 1, sharex=True, figsize=(4, 7))

ax = axs[0]
ax.plot(vs, mods_stretch, label='stretch')
ax.plot(vs, mods_release, label='release')
ax.plot(vs, mods_avg, label='avg')
ax.set_ylabel("E [?]")
ax.legend()

mods_stretch_rel = mods_stretch/mods_stretch[0]
mods_release_rel = mods_release/mods_release[0]
mods_avg_rel = mods_avg/mods_avg[0]
ax = axs[1]
ax.plot(vs, mods_stretch_rel, label='stretch')
ax.plot(vs, mods_release_rel, label='release')
ax.plot(vs, mods_avg_rel, label='avg')
ax.set_ylabel("$E/E_{v=1}$")


mslope_stretch = (mods_stretch_rel-1)/(vs-1)
mslope_release = (mods_release_rel-1)/(vs-1)
mslope_avg = (mods_avg_rel-1)/(vs-1)
ax = axs[2]
ax.plot(vs, mslope_stretch, label='stretch')
ax.plot(vs, mslope_release, label='release')
ax.plot(vs, mslope_avg, label='avg')
ax.set_ylabel(r"$m'=\frac{d E/E_{v=1}}{dv}$")
ax.set_yticks(np.linspace(-4, 0, 11))
# ax.set_ylim(-4, 0)
ax.grid()

axs[-1].set_xlabel("v")

# The nominal m' at dehydration level `i` is:
# Note 1 -> 40%, 2 -> 60%, etc.
i = 1
(mods_avg_rel[i]-mods_avg_rel[0])/(vs[i]-vs[0])